<a href="https://colab.research.google.com/github/Seghelicious/Vayner-Box/blob/master/Vayner_Box_Asset_Portfolio_Returns%2C_Regression_and_Monte_Carlo_Simulation_based_on_the_Brownian_Motion_of_Financial_Markets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Mean - Variance Optimisation**

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [0]:
# Import the required libraries and packages

import pandas as pd
import numpy as np
import math
import itertools
import warnings
warnings.filterwarnings('ignore')
import datetime as dt
from datetime import datetime
from scipy import stats
from scipy.stats import norm
import statsmodels.api as sm

In [0]:
# ead csv into colab
fund_prices = pd.read_csv('/content/drive/My Drive/Python University/Vayner Box/fund_prices (version 1).csv')
axa_mmf_ = pd.read_csv('/content/drive/My Drive/Python University/Vayner Box/axa_mmf1.csv')

In [0]:
# Reset date format
fund_prices['Date'] = pd.to_datetime(fund_prices.Date, format = '%Y-%m-%d') 
axa_mmf_['Date'] = pd.to_datetime(axa_mmf_.Date, format = '%Y-%m-%d')

In [0]:
# Calculate the daily yields
axa_mmf_['AXA Money Market Fund Yield'] = (1 + axa_mmf_['AXA Money Market Fund Yield']) ** (1/365) - 1

In [0]:
# Summarize the datetime format to (Year-Month)
axa_mmf_['month year'] = axa_mmf_['Date'].apply(lambda x: x.strftime('%Y-%m'))
axa_mmf_ = axa_mmf_.drop(columns = ['Date'], axis = 1)

In [0]:
# Calculate the monthly returns from daily yields
axa_mmf_ = axa_mmf_.groupby('month year')['AXA Money Market Fund Yield'].sum().reset_index()

#### **Monthly Returns**

In [0]:
# Create a copy of the dataset

fund_prices = fund_prices.copy()

In [0]:
# extract some date information

fund_prices['year'] = fund_prices['Date'].dt.year
fund_prices['month'] = fund_prices['Date'].dt.month
fund_prices['day'] = fund_prices['Date'].dt.day
fund_prices['day of month'] = fund_prices['Date'].dt.daysinmonth

In [0]:
# Extract the first and last days of each month

first_day28 = fund_prices[(fund_prices['day'] == 1) & (fund_prices['day of month'] == 28)]
last_day28 = fund_prices[(fund_prices['day'] == 28) & (fund_prices['day of month'] == 28)]
first_day30 = fund_prices[(fund_prices['day'] == 1) & (fund_prices['day of month'] == 30)]
last_day30 = fund_prices[(fund_prices['day'] == 30) & (fund_prices['day of month'] == 30)]
first_day31 = fund_prices[(fund_prices['day'] == 1) & (fund_prices['day of month'] == 31)]
last_day31 = fund_prices[(fund_prices['day'] == 31) & (fund_prices['day of month'] == 31)]

In [0]:
# Concatenate the first and last days rows to form new dataset

new_prices = pd.concat([first_day28, last_day28, first_day30, last_day30, first_day31, last_day31], axis = 0)

In [0]:
# Summarise the datetime format to (Year-Month) and rearrange the index

new_prices['month year'] = new_prices['Date'].apply(lambda x: x.strftime('%Y-%m'))
new_prices = new_prices.sort_index(ascending = True)

In [0]:
# Calculate the percentage change between the first and last days

stb_etf_returns = new_prices['Stanbic ETF 30'].div(new_prices.groupby('month year')['Stanbic ETF 30'].shift(1)) - 1
stb_abs_returns = new_prices['Stanbic Absolute Fund'].div(new_prices.groupby('month year')['Stanbic Absolute Fund'].shift(1)) - 1
stb_gtd_returns = new_prices['Stanbic Guaranteed Income Fund'].div(new_prices.groupby('month year')['Stanbic Guaranteed Income Fund'].shift(1)) - 1
stb_euro_returns = new_prices['Stanbic Eurobond Fund'].div(new_prices.groupby('month year')['Stanbic Eurobond Fund'].shift(1)) - 1
fbn_euro_returns = new_prices['FBN Eurobond Fund'].div(new_prices.groupby('month year')['FBN Eurobond Fund'].shift(1)) - 1

In [0]:
# Create new dataframe from the %change values

monthly_returns = pd.DataFrame()

monthly_returns['Date'] = new_prices['month year']
monthly_returns['Stanbic ETF 30'] = stb_etf_returns
monthly_returns['Stanbic Absolute Fund'] = stb_abs_returns
monthly_returns['Stanbic Guaranteed Income Fund'] = stb_gtd_returns
monthly_returns['Stanbic Eurobond Fund'] = stb_euro_returns
monthly_returns['FBN Eurobond Fund'] = fbn_euro_returns

In [0]:
# Concatenate the AXA MMF monthly yields values to the new monthly_returns dataframe

monthly_returns = monthly_returns.dropna()
monthly_returns = monthly_returns.reset_index(drop = True, inplace = False)
month_returns = pd.concat([monthly_returns, axa_mmf_['AXA Money Market Fund Yield']], axis = 1)

#### **Portfolio Returns**

Calculate Portfolio Expected Returns

In [0]:
returns1 = month_returns[['Stanbic Absolute Fund', 'AXA Money Market Fund Yield']]
returns2 = month_returns[['Stanbic Absolute Fund', 'Stanbic Guaranteed Income Fund', 'AXA Money Market Fund Yield']]            
returns3 = month_returns[['Stanbic Absolute Fund', 'Stanbic ETF 30']]
returns4 = month_returns[['Stanbic Eurobond Fund', 'FBN Eurobond Fund']]

In [0]:
axa_mmf1, axa_mmf2, axa_mmf3 = [0.9, 0.72, 0.13]
stb_abs1, stb_abs2, stb_abs3, stb_abs4, stb_abs5 = [0.1, 0.24, 0.87, 0.73, 0.51]
stb_GI = 0.04
stb_etf1, stb_etf2 = [0.27, 0.49]
stb_euro, fbn_euro = [0.5, 0.5]

In [0]:
conserv = np.array([stb_abs1, axa_mmf1])
caut = np.array([stb_abs2, stb_GI, axa_mmf2])
bal = np.array([stb_abs3, axa_mmf3])
grow = np.array([stb_abs4, stb_etf1])
aggres = np.array([stb_abs5, stb_etf2])
doll = np.array([stb_euro, fbn_euro])

In [0]:
conservative = np.dot(returns1, conserv)
cautious = np.dot(returns2, caut)
balanced = np.dot(returns1, bal)
growth = np.dot(returns3, grow)
aggressive = np.dot(returns3, aggres)
dollar = np.dot(returns4, doll)

In [0]:
# Create dataframe from calculated portfolio returns
portfolios = pd.DataFrame()

portfolios['Date'] = month_returns['Date']
portfolios['Conservative'] = conservative
portfolios['Cautious'] = cautious
portfolios['Balanced'] = balanced
portfolios['Growth'] = growth
portfolios['Aggressive'] = aggressive
portfolios['Dollar'] = dollar

In [21]:
portfolios.head()

,Date,Conservative,Cautious,Balanced,Growth,Aggressive,Dollar
0,2016-08,0.006032,0.006028,0.006457,0.004061,0.002050,0.005263
1,2016-09,0.006469,0.006824,0.008787,0.007953,0.006955,-0.000189
2,2016-10,0.009715,0.010810,0.014668,0.000713,-0.011338,-0.014401
3,2016-11,0.011753,0.012317,0.014148,-0.006651,-0.023928,-0.002098
4,2016-12,0.012194,0.012592,0.013947,0.024315,0.032521,0.010129


#### **Regression**

In [0]:
col_names = ['Oil Prices', 'Inflation', 'NSE ASI', 'FX Reserves', 'USD/NGN (BDC)', 
             'USD/NGN (I&E)', '91-Day Yield', '182-Day Yield', '364-Day Yield', 
             '2yr-Bond Yield', '3yr-Bond Yield', '5-yr-Bond Yield', '7yr-Bond Yield', 
             '10yr-Bond Yield', '12yr-Bond Yield', '15yr-Bond Yield', '20yr-Bond Yield']

# Read macrodata dataset into notebook
macro_data = pd.read_csv('/content/drive/My Drive/Python University/Vayner Box/Portfolios & Macro Data.csv', usecols = col_names)

In [0]:
data = pd.concat([portfolios, macro_data], axis = 1)

In [0]:
# Independent Variables
X1 = data[['Oil Prices', 'Inflation', 'NSE ASI', 'FX Reserves', 'USD/NGN (BDC)', 'USD/NGN (I&E)', '91-Day Yield', 
           '182-Day Yield', '364-Day Yield', '2yr-Bond Yield', '3yr-Bond Yield', '5-yr-Bond Yield', '7yr-Bond Yield', 
           '10yr-Bond Yield', '12yr-Bond Yield', '15yr-Bond Yield', '20yr-Bond Yield']]

##### *Conservative Portfolio*

In [25]:
# Dependent Variable
Y1 = data['Conservative']

X1a = sm.add_constant(X1)
reg1 = sm.OLS(Y1, X1a).fit()
#prediction1 = reg1.predict(X1)

reg1.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           Conservative   R-squared:                       0.734
Model:                            OLS   Adj. R-squared:                  0.281
Method:                 Least Squares   F-statistic:                     1.621
Date:                Sat, 16 May 2020   Prob (F-statistic):              0.220
Time:                        17:26:34   Log-Likelihood:                 148.26
No. Observations:                  28   AIC:                            -260.5
Df Residuals:                      10   BIC:                            -236.5
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.0107      0.001     19.743      0.000       0.010       0.012
Oil Prices          0.0061      0.008      0.775      0.456      -0.011       0.024
Inflation          -0.0051      0.018     -0.285      0.782      -0.045       0.035
NSE ASI             0.0207      0.008      2.731      0.021       0.004       0.038
FX Reserves         0.0251      0.013      1.986      0.075      -0.003       0.053
USD/NGN (BDC)      -0.0189      0.011     -1.671      0.126      -0.044       0.006
USD/NGN (I&E)       0.0094      0.010      0.914      0.382      -0.013       0.032
91-Day Yield        0.0006      0.005      0.117      0.909      -0.011       0.013
182-Day Yield      -0.0033      0.009     -0.352      0.732      -0.024       0.018
364-Day Yield       0.0007      0.012      0.058      0.955      -0.026       0.027
2yr-Bond Yield     -0.0062      0.015     -0.410      0.690      -0.040       0.027
3yr-Bond Yield      0.0208      0.031      0.674      0.516      -0.048       0.089
5-yr-Bond Yield     0.0049      0.048      0.102      0.920      -0.102       0.112
7yr-Bond Yield     -0.0199      0.072     -0.277      0.787      -0.180       0.140
10yr-Bond Yield    -0.0041      0.087     -0.047      0.963      -0.197       0.189
12yr-Bond Yield     0.0663      0.160      0.415      0.687      -0.290       0.422
15yr-Bond Yield    -0.0406      0.112     -0.362      0.725      -0.291       0.210
20yr-Bond Yield    -0.0209      0.058     -0.358      0.728      -0.151       0.109
==============================================================================
Omnibus:                        4.125   Durbin-Watson:                   1.533
Prob(Omnibus):                  0.127   Jarque-Bera (JB):                2.519
Skew:                           0.500   Prob(JB):                        0.284
Kurtosis:                       4.076   Cond. No.                         514.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [26]:
# Independent Variables
X1_conserv = data[['NSE ASI', 'FX Reserves']]

# Dependent Variable
Y1_conserv = data['Conservative']

X1a_conserv = sm.add_constant(X1_conserv)
reg1_conserv = sm.OLS(Y1_conserv, X1a_conserv).fit()
#prediction1 = reg1.predict(X1)

reg1_conserv.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           Conservative   R-squared:                       0.408
Model:                            OLS   Adj. R-squared:                  0.361
Method:                 Least Squares   F-statistic:                     8.632
Date:                Sat, 16 May 2020   Prob (F-statistic):            0.00141
Time:                        17:26:34   Log-Likelihood:                 137.08
No. Observations:                  28   AIC:                            -268.2
Df Residuals:                      25   BIC:                            -264.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.0110      0.000     27.597      0.000       0.010       0.012
NSE ASI         0.0166      0.006      2.866      0.008       0.005       0.029
FX Reserves     0.0250      0.008      2.961      0.007       0.008       0.042
==============================================================================
Omnibus:                        1.794   Durbin-Watson:                   1.029
Prob(Omnibus):                  0.408   Jarque-Bera (JB):                1.288
Skew:                          -0.522   Prob(JB):                        0.525
Kurtosis:                       2.882   Cond. No.                         23.4
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

##### *Cautious Portfolio*

In [27]:
Y2 = data['Cautious']

X1b = sm.add_constant(X1)
reg2 = sm.OLS(Y2, X1b).fit()
#prediction1 = reg1.predict(X1)

reg2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Cautious   R-squared:                       0.743
Model:                            OLS   Adj. R-squared:                  0.306
Method:                 Least Squares   F-statistic:                     1.701
Date:                Sat, 16 May 2020   Prob (F-statistic):              0.197
Time:                        17:26:34   Log-Likelihood:                 148.24
No. Observations:                  28   AIC:                            -260.5
Df Residuals:                      10   BIC:                            -236.5
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.0109      0.001     20.009      0.000       0.010       0.012
Oil Prices          0.0049      0.008      0.621      0.549      -0.013       0.023
Inflation          -0.0026      0.018     -0.143      0.889      -0.042       0.037
NSE ASI             0.0211      0.008      2.787      0.019       0.004       0.038
FX Reserves         0.0257      0.013      2.036      0.069      -0.002       0.054
USD/NGN (BDC)      -0.0195      0.011     -1.724      0.115      -0.045       0.006
USD/NGN (I&E)       0.0098      0.010      0.954      0.363      -0.013       0.033
91-Day Yield        0.0013      0.005      0.237      0.817      -0.011       0.013
182-Day Yield      -0.0034      0.009     -0.360      0.726      -0.024       0.017
364-Day Yield      -0.0033      0.012     -0.277      0.788      -0.030       0.023
2yr-Bond Yield     -0.0077      0.015     -0.513      0.619      -0.041       0.026
3yr-Bond Yield      0.0283      0.031      0.918      0.380      -0.040       0.097
5-yr-Bond Yield     0.0119      0.048      0.247      0.810      -0.095       0.119
7yr-Bond Yield     -0.0443      0.072     -0.617      0.551      -0.204       0.116
10yr-Bond Yield    -0.0093      0.087     -0.107      0.917      -0.203       0.184
12yr-Bond Yield     0.0452      0.160      0.283      0.783      -0.311       0.401
15yr-Bond Yield    -0.0144      0.112     -0.128      0.900      -0.265       0.236
20yr-Bond Yield    -0.0040      0.058     -0.068      0.947      -0.134       0.126
==============================================================================
Omnibus:                        4.636   Durbin-Watson:                   1.723
Prob(Omnibus):                  0.098   Jarque-Bera (JB):                2.945
Skew:                           0.579   Prob(JB):                        0.229
Kurtosis:                       4.087   Cond. No.                         514.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [28]:
# Independent Variables
X1_caut = data[['NSE ASI', 'FX Reserves']]

# Dependent Variable
Y2_caut = data['Cautious']


X1b_caut = sm.add_constant(X1_caut)
reg2_caut = sm.OLS(Y2_caut, X1b_caut).fit()
#prediction1 = reg1.predict(X1)

reg2_caut.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Cautious   R-squared:                       0.415
Model:                            OLS   Adj. R-squared:                  0.369
Method:                 Least Squares   F-statistic:                     8.881
Date:                Sat, 16 May 2020   Prob (F-statistic):            0.00122
Time:                        17:26:34   Log-Likelihood:                 136.73
No. Observations:                  28   AIC:                            -267.5
Df Residuals:                      25   BIC:                            -263.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.0112      0.000     27.547      0.000       0.010       0.012
NSE ASI         0.0172      0.006      2.939      0.007       0.005       0.029
FX Reserves     0.0254      0.009      2.973      0.006       0.008       0.043
==============================================================================
Omnibus:                        1.581   Durbin-Watson:                   1.100
Prob(Omnibus):                  0.454   Jarque-Bera (JB):                1.023
Skew:                          -0.468   Prob(JB):                        0.600
Kurtosis:                       2.965   Cond. No.                         23.4
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

##### *Balanced Portfolio*

In [29]:
Y3 = data['Balanced']


X1c = sm.add_constant(X1)
reg3 = sm.OLS(Y3, X1c).fit()
#prediction1 = reg1.predict(X1)

reg3.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Balanced   R-squared:                       0.657
Model:                            OLS   Adj. R-squared:                  0.073
Method:                 Least Squares   F-statistic:                     1.126
Date:                Sat, 16 May 2020   Prob (F-statistic):              0.438
Time:                        17:26:34   Log-Likelihood:                 142.29
No. Observations:                  28   AIC:                            -248.6
Df Residuals:                      10   BIC:                            -224.6
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.0115      0.001     17.171      0.000       0.010       0.013
Oil Prices          0.0030      0.010      0.304      0.768      -0.019       0.025
Inflation           0.0037      0.022      0.168      0.870      -0.045       0.053
NSE ASI             0.0202      0.009      2.158      0.056      -0.001       0.041
FX Reserves         0.0230      0.016      1.475      0.171      -0.012       0.058
USD/NGN (BDC)      -0.0266      0.014     -1.899      0.087      -0.058       0.005
USD/NGN (I&E)       0.0138      0.013      1.084      0.304      -0.015       0.042
91-Day Yield        0.0020      0.007      0.306      0.766      -0.013       0.017
182-Day Yield      -0.0006      0.012     -0.051      0.960      -0.026       0.025
364-Day Yield      -0.0066      0.015     -0.445      0.666      -0.039       0.026
2yr-Bond Yield     -0.0167      0.019     -0.899      0.390      -0.058       0.025
3yr-Bond Yield      0.0211      0.038      0.553      0.592      -0.064       0.106
5-yr-Bond Yield     0.0477      0.059      0.803      0.441      -0.085       0.180
7yr-Bond Yield     -0.1057      0.089     -1.192      0.261      -0.303       0.092
10yr-Bond Yield    -0.0191      0.107     -0.178      0.863      -0.258       0.220
12yr-Bond Yield     0.0815      0.198      0.412      0.689      -0.359       0.522
15yr-Bond Yield    -0.0340      0.139     -0.245      0.812      -0.344       0.276
20yr-Bond Yield     0.0360      0.072      0.497      0.630      -0.125       0.197
==============================================================================
Omnibus:                        2.850   Durbin-Watson:                   2.133
Prob(Omnibus):                  0.240   Jarque-Bera (JB):                1.663
Skew:                           0.572   Prob(JB):                        0.435
Kurtosis:                       3.340   Cond. No.                         514.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [30]:
# Independent Variables
X1_bal = data[['NSE ASI', 'USD/NGN (BDC)']]

# Dependent Variable
Y2_bal = data['Balanced']


X1b_bal = sm.add_constant(X1_bal)
reg2_bal = sm.OLS(Y2_bal, X1b_bal).fit()
#prediction1 = reg1.predict(X1)

reg2_bal.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               Balanced   R-squared:                       0.272
Model:                            OLS   Adj. R-squared:                  0.213
Method:                 Least Squares   F-statistic:                     4.661
Date:                Sat, 16 May 2020   Prob (F-statistic):             0.0190
Time:                        17:26:34   Log-Likelihood:                 131.75
No. Observations:                  28   AIC:                            -257.5
Df Residuals:                      25   BIC:                            -253.5
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const             0.0123      0.000     27.929      0.000       0.011       0.013
NSE ASI           0.0179      0.007      2.536      0.018       0.003       0.032
USD/NGN (BDC)    -0.0132      0.007     -2.025      0.054      -0.027       0.000
==============================================================================
Omnibus:                        0.435   Durbin-Watson:                   1.088
Prob(Omnibus):                  0.804   Jarque-Bera (JB):                0.555
Skew:                          -0.016   Prob(JB):                        0.758
Kurtosis:                       2.311   Cond. No.                         16.7
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

##### *Growth Portfolio*

In [31]:
Y4 = data['Growth']


X1d = sm.add_constant(X1)
reg4 = sm.OLS(Y4, X1d).fit()
#prediction1 = reg1.predict(X1)

reg4.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Growth   R-squared:                       0.874
Model:                            OLS   Adj. R-squared:                  0.659
Method:                 Least Squares   F-statistic:                     4.075
Date:                Sat, 16 May 2020   Prob (F-statistic):             0.0142
Time:                        17:26:34   Log-Likelihood:                 100.12
No. Observations:                  28   AIC:                            -164.2
Df Residuals:                      10   BIC:                            -140.3
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.0063      0.003      2.068      0.065      -0.000       0.013
Oil Prices          0.0179      0.044      0.407      0.693      -0.080       0.116
Inflation          -0.0518      0.099     -0.520      0.614      -0.273       0.170
NSE ASI             0.2627      0.042      6.220      0.000       0.169       0.357
FX Reserves        -0.0136      0.070     -0.194      0.850      -0.171       0.143
USD/NGN (BDC)      -0.0095      0.063     -0.151      0.883      -0.150       0.131
USD/NGN (I&E)       0.0859      0.057      1.501      0.164      -0.042       0.213
91-Day Yield        0.0026      0.030      0.087      0.933      -0.064       0.069
182-Day Yield      -0.0086      0.052     -0.166      0.872      -0.125       0.108
364-Day Yield      -0.0657      0.067     -0.987      0.347      -0.214       0.083
2yr-Bond Yield     -0.0825      0.084     -0.983      0.349      -0.270       0.105
3yr-Bond Yield     -0.0353      0.172     -0.205      0.842      -0.418       0.348
5-yr-Bond Yield    -0.0374      0.268     -0.140      0.892      -0.635       0.560
7yr-Bond Yield     -0.0265      0.400     -0.066      0.948      -0.918       0.865
10yr-Bond Yield     0.4843      0.484      1.000      0.341      -0.595       1.563
12yr-Bond Yield     0.0156      0.891      0.018      0.986      -1.970       2.001
15yr-Bond Yield     0.1068      0.627      0.170      0.868      -1.290       1.504
20yr-Bond Yield    -0.3583      0.326     -1.098      0.298      -1.085       0.369
==============================================================================
Omnibus:                        6.384   Durbin-Watson:                   3.153
Prob(Omnibus):                  0.041   Jarque-Bera (JB):                5.207
Skew:                          -0.598   Prob(JB):                       0.0740
Kurtosis:                       4.741   Cond. No.                         514.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

##### Aggressive Portfolio

In [32]:
Y5 = data['Aggressive']


X1e = sm.add_constant(X1)
reg5 = sm.OLS(Y5, X1e).fit()
#prediction1 = reg1.predict(X1)

reg5.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             Aggressive   R-squared:                       0.860
Model:                            OLS   Adj. R-squared:                  0.623
Method:                 Least Squares   F-statistic:                     3.619
Date:                Sat, 16 May 2020   Prob (F-statistic):             0.0217
Time:                        17:26:34   Log-Likelihood:                 82.632
No. Observations:                  28   AIC:                            -129.3
Df Residuals:                      10   BIC:                            -105.3
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.0019      0.006      0.328      0.749      -0.011       0.014
Oil Prices          0.0306      0.082      0.371      0.718      -0.153       0.214
Inflation          -0.0982      0.186     -0.528      0.609      -0.512       0.316
NSE ASI             0.4604      0.079      5.836      0.000       0.285       0.636
FX Reserves        -0.0433      0.132     -0.329      0.749      -0.336       0.250
USD/NGN (BDC)       0.0054      0.118      0.046      0.964      -0.258       0.269
USD/NGN (I&E)       0.1441      0.107      1.348      0.207      -0.094       0.382
91-Day Yield        0.0028      0.056      0.051      0.960      -0.121       0.127
182-Day Yield      -0.0156      0.098     -0.160      0.876      -0.233       0.202
364-Day Yield      -0.1130      0.124     -0.908      0.385      -0.390       0.164
2yr-Bond Yield     -0.1347      0.157     -0.859      0.411      -0.484       0.215
3yr-Bond Yield     -0.0813      0.321     -0.253      0.805      -0.797       0.634
5-yr-Bond Yield    -0.1126      0.501     -0.225      0.827      -1.228       1.003
7yr-Bond Yield      0.0499      0.747      0.067      0.948      -1.615       1.714
10yr-Bond Yield     0.8966      0.904      0.991      0.345      -1.119       2.912
12yr-Bond Yield    -0.0402      1.664     -0.024      0.981      -3.749       3.668
15yr-Bond Yield     0.2206      1.171      0.188      0.854      -2.389       2.830
20yr-Bond Yield    -0.6874      0.609     -1.128      0.285      -2.045       0.670
==============================================================================
Omnibus:                        5.027   Durbin-Watson:                   3.120
Prob(Omnibus):                  0.081   Jarque-Bera (JB):                3.517
Skew:                          -0.522   Prob(JB):                        0.172
Kurtosis:                       4.388   Cond. No.                         514.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

##### *Dollar Portfolio*

In [33]:
Y6 = data['Dollar']


X1f = sm.add_constant(X1)
reg6 = sm.OLS(Y6, X1f).fit()
#prediction1 = reg1.predict(X1)

reg6.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 Dollar   R-squared:                       0.599
Model:                            OLS   Adj. R-squared:                 -0.083
Method:                 Least Squares   F-statistic:                    0.8784
Date:                Sat, 16 May 2020   Prob (F-statistic):              0.609
Time:                        17:26:35   Log-Likelihood:                 116.23
No. Observations:                  28   AIC:                            -196.5
Df Residuals:                      10   BIC:                            -172.5
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
const               0.0016      0.002      0.921      0.379      -0.002       0.005
Oil Prices          0.0131      0.025      0.526      0.610      -0.042       0.068
Inflation          -0.0578      0.056     -1.034      0.326      -0.183       0.067
NSE ASI             0.0350      0.024      1.474      0.171      -0.018       0.088
FX Reserves         0.0170      0.040      0.428      0.678      -0.071       0.105
USD/NGN (BDC)       0.0147      0.036      0.413      0.689      -0.065       0.094
USD/NGN (I&E)       0.0231      0.032      0.718      0.489      -0.049       0.095
91-Day Yield        0.0025      0.017      0.149      0.884      -0.035       0.040
182-Day Yield       0.0220      0.029      0.750      0.471      -0.043       0.087
364-Day Yield       0.0258      0.037      0.689      0.507      -0.058       0.109
2yr-Bond Yield      0.0018      0.047      0.037      0.971      -0.104       0.107
3yr-Bond Yield     -0.0034      0.097     -0.036      0.972      -0.219       0.212
5-yr-Bond Yield     0.0764      0.151      0.507      0.623      -0.260       0.412
7yr-Bond Yield      0.0379      0.225      0.169      0.869      -0.463       0.539
10yr-Bond Yield    -0.4285      0.272     -1.573      0.147      -1.035       0.178
12yr-Bond Yield     0.3311      0.501      0.660      0.524      -0.786       1.448
15yr-Bond Yield    -0.0400      0.353     -0.113      0.912      -0.826       0.746
20yr-Bond Yield    -0.0477      0.183     -0.260      0.800      -0.457       0.361
==============================================================================
Omnibus:                        3.245   Durbin-Watson:                   1.947
Prob(Omnibus):                  0.197   Jarque-Bera (JB):                1.899
Skew:                          -0.597   Prob(JB):                        0.387
Kurtosis:                       3.453   Cond. No.                         514.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

### **Monte Carlo Simulation**

In [0]:
# Calculate logarithmic returns from monthly asset returns

stb_etf_returns1 = (new_prices['Stanbic ETF 30'].div(new_prices.groupby('month year')['Stanbic ETF 30'].shift(1)) - 1).dropna().reset_index(drop = True)
stb_abs_returns1 = (new_prices['Stanbic Absolute Fund'].div(new_prices.groupby('month year')['Stanbic Absolute Fund'].shift(1)) - 1).dropna().reset_index(drop = True)
stb_gtd_returns1 = (new_prices['Stanbic Guaranteed Income Fund'].div(new_prices.groupby('month year')['Stanbic Guaranteed Income Fund'].shift(1)) - 1).dropna().reset_index(drop = True)
stb_euro_returns1 = (new_prices['Stanbic Eurobond Fund'].div(new_prices.groupby('month year')['Stanbic Eurobond Fund'].shift(1)) - 1).dropna().reset_index(drop = True)
fbn_euro_returns1 = (new_prices['FBN Eurobond Fund'].div(new_prices.groupby('month year')['FBN Eurobond Fund'].shift(1)) - 1).dropna().reset_index(drop = True)
axa_mmf_returns1 = (axa_mmf_['AXA Money Market Fund Yield']).reset_index(drop = True)

In [0]:
# Create a dataframe from log returns of asset

month_returns_log = pd.DataFrame({'Date': month_returns['Date'], 
                                  'Stanbic ETF 30': stb_etf_returns1, 
                                  'Stanbic Absolute Fund': stb_abs_returns1, 
                                  'Stanbic Guaranteed Income Fund': stb_gtd_returns1, 
                                  'Stanbic Eurobond Fund': stb_euro_returns1, 
                                  'FBN Eurobond Fund': fbn_euro_returns1, 
                                  'AXA Money Market Fund Yield': axa_mmf_returns1})                                 

In [0]:
# Calculate the average return of each asset

stb_etf_mean = np.array(month_returns_log['Stanbic ETF 30'].mean())
stb_abs_mean = np.array(month_returns_log['Stanbic Absolute Fund'].mean())
stb_gtb_mean = np.array(month_returns_log['Stanbic Guaranteed Income Fund'].mean())
stb_euro_mean = np.array(month_returns_log['Stanbic Eurobond Fund'].mean())
fbn_euro_mean = np.array(month_returns_log['FBN Eurobond Fund'].mean())
axa_mmf_mean = np.array(month_returns_log['AXA Money Market Fund Yield'].mean())

In [0]:
# Calculate the variance of each asset

stb_etf_var = np.array(month_returns_log['Stanbic ETF 30'].var())
stb_abs_var = np.array(month_returns_log['Stanbic Absolute Fund'].var())
stb_gtb_var = np.array(month_returns_log['Stanbic Guaranteed Income Fund'].var())
stb_euro_var = np.array(month_returns_log['Stanbic Eurobond Fund'].var())
fbn_euro_var = np.array(month_returns_log['FBN Eurobond Fund'].var())
axa_mmf_var = np.array(month_returns_log['AXA Money Market Fund Yield'].var())

In [0]:
# Calculate the standard deviation of each asset

stb_etf_std = np.array(month_returns_log['Stanbic ETF 30'].std())
stb_abs_std = np.array(month_returns_log['Stanbic Absolute Fund'].std())
stb_gtb_std = np.array(month_returns_log['Stanbic Guaranteed Income Fund'].std())
stb_euro_std = np.array(month_returns_log['Stanbic Eurobond Fund'].std())
fbn_euro_std = np.array(month_returns_log['FBN Eurobond Fund'].std())
axa_mmf_std = np.array(month_returns_log['AXA Money Market Fund Yield'].std())

In [0]:
# Calculate the drift of each asset

stb_etf_drift = np.array(stb_etf_mean - ( 0.5 * stb_etf_var))
stb_abs_drift = np.array(stb_abs_mean - ( 0.5 * stb_abs_var))
stb_gtb_drift = np.array(stb_gtb_mean - ( 0.5 * stb_gtb_var))
stb_euro_drift = np.array(stb_euro_mean - ( 0.5 * stb_euro_var))
fbn_euro_drift = np.array(fbn_euro_mean - ( 0.5 * fbn_euro_var))
axa_mmf_drift = np.array(axa_mmf_mean - ( 0.5 * axa_mmf_var))

In [0]:
t_intervals = 1200
n_iterations = 1000

In [0]:
# Calculates the predicted returns using the Brownian Motion for Fianacial Markets model

stb_etf_pred = np.exp(stb_etf_drift + stb_etf_std * norm.ppf(np.random.rand(t_intervals, n_iterations)))
stb_abs_pred = np.exp(stb_abs_drift + stb_abs_std * norm.ppf(np.random.rand(t_intervals, n_iterations)))
stb_gtb_pred = np.exp(stb_gtb_drift + stb_gtb_std * norm.ppf(np.random.rand(t_intervals, n_iterations)))
stb_euro_pred = np.exp(stb_euro_drift + stb_euro_std * norm.ppf(np.random.rand(t_intervals, n_iterations)))
fbn_euro_pred = np.exp(fbn_euro_drift + fbn_euro_std * norm.ppf(np.random.rand(t_intervals, n_iterations)))
axa_mmf_pred = np.exp(axa_mmf_drift + axa_mmf_std * norm.ppf(np.random.rand(t_intervals, n_iterations)))

In [0]:
# Create a dataframe from each assets returns

stb_etf_pred = pd.DataFrame(data = stb_etf_pred[:,:])
stb_abs_pred = pd.DataFrame(data = stb_abs_pred[:,:])
stb_gtb_pred = pd.DataFrame(data = stb_gtb_pred[:,:])
stb_euro_pred = pd.DataFrame(data = stb_euro_pred[:,:])
fbn_euro_pred = pd.DataFrame(data = fbn_euro_pred[:,:])
axa_mmf_pred = pd.DataFrame(data = axa_mmf_pred[:,:])

In [0]:
#future_dates = pd.date_range(start = '2018-11', periods = 1200, freq = 'M')
#future_dates.strftime('%Y-%m')
#index = 0

###### *Conservative Portfolio Simulation*

In [44]:
# Calculates portfolio expected returns from the portfolio ratios and the predicted returns 

stb_a = stb_abs_pred * stb_abs1
axa_a = axa_mmf_pred * axa_mmf1

# 100000 is the invested amount
conserv = stb_a.add(axa_a)
conservative =  conserv * 100000

#conservative.insert(loc = index, column = 'Date', value = future_dates.strftime('%Y-%m'))

conserv.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
0,1.012398,1.010089,1.013256,1.012928,1.012005,1.012273,1.007402,1.008497,1.012760,1.012560,1.010344,1.010260,1.012217,1.012272,1.011390,1.013632,1.010265,1.011743,1.015401,1.010493,1.010194,1.010963,1.011757,1.011013,1.009341,1.011766,1.013289,1.012691,1.014557,1.014010,1.010254,1.013302,1.010250,1.015797,1.014692,1.009724,1.012854,1.012488,1.011009,1.008865,...,1.012935,1.009856,1.012825,1.013203,1.011806,1.013498,1.010423,1.013868,1.013638,1.008779,1.010629,1.010199,1.012774,1.013690,1.010807,1.014094,1.013494,1.011582,1.009609,1.012197,1.015158,1.011272,1.011233,1.008659,1.010451,1.013438,1.013017,1.012220,1.013133,1.014987,1.012970,1.011138,1.014250,1.013647,1.007504,1.006392,1.011316,1.015306,1.007566,1.009731
1,1.012667,1.014375,1.009889,1.010006,1.010825,1.013577,1.013156,1.011643,1.008440,1.011566,1.009410,1.014865,1.008377,1.009707,1.011435,1.008672,1.009123,1.014004,1.013908,1.012678,1.012828,1.010042,1.007641,1.007949,1.013936,1.009831,1.010516,1.013293,1.012037,1.007871,1.011497,1.011584,1.012812,1.004689,1.012183,1.010692,1.011580,1.009064,1.011032,1.013841,...,1.009912,1.008046,1.011217,1.008359,1.011760,1.007597,1.007575,1.013361,1.011134,1.016523,1.014833,1.014339,1.006598,1.011105,1.014551,1.008247,1.010129,1.011198,1.014598,1.009716,1.011038,1.013893,1.013442,1.009332,1.008711,1.011701,1.012577,1.009434,1.014679,1.011629,1.012084,1.013405,1.011660,1.017027,1.012550,1.009126,1.013248,1.011321,1.012023,1.010319
2,1.009401,1.011799,1.009260,1.014298,1.011008,1.009731,1.011639,1.009315,1.009824,1.016171,1.011790,1.008575,1.008775,1.006918,1.014805,1.011296,1.011924,1.009285,1.012845,1.013618,1.010955,1.012938,1.012344,1.010940,1.011723,1.006315,1.010729,1.013552,1.006988,1.007998,1.009269,1.012151,1.011574,1.006129,1.012397,1.013401,1.010261,1.014313,1.010058,1.012992,...,1.011643,1.007598,1.016199,1.015023,1.013622,1.012787,1.012748,1.009890,1.014378,1.009031,1.014962,1.009212,1.007244,1.012491,1.012551,1.014681,1.011495,1.014371,1.014851,1.008901,1.012549,1.014375,1.010983,1.007036,1.010052,1.012380,1.012051,1.011947,1.011680,1.007459,1.015500,1.010949,1.012670,1.009848,1.011934,1.014009,1.012538,1.009929,1.014123,1.007402
3,1.009948,1.009411,1.012134,1.011716,1.012504,1.006652,1.008535,1.012388,1.009523,1.013484,1.010427,1.011793,1.009209,1.010910,1.011504,1.015062,1.012340,1.010858,1.012818,1.013365,1.012571,1.010680,1.012827,1.008040,1.011057,1.012882,1.012682,1.015280,1.011310,1.017962,1.009591,1.010289,1.014126,1.010175,1.013112,1.008454,1.014741,1.008672,1.010134,1.009791,...,1.013200,1.014370,1.012358,1.015601,1.010385,1.012721,1.009569,1.013553,1.009149,1.011789,1.012604,1.013768,1.012530,1.012023,1.011947,1.008121,1.012375,1.014365,1.014695,1.013685,1.010968,1.012996,1.010843,1.009746,1.011781,1.013403,1.011313,1.011544,1.009014,1.012495,1.010691,1.013447,1.010480,1.011221,1.012589,1.014758,1.013627,1.011819,1.012443,1.010500
4,1.012313,1.010916,1.013077,1.005784,1.015030,1.009441,1.009650,1.013173,1.013814,1.013554,1.010424,1.009940,1.006437,1.011168,1.007555,1.010682,1.011954,1.011727,1.008967,1.011027,1.011865,1.009730,1.011177,1.012024,1.010023,1.008839,1.010556,1.013096,1.013022,1.011151,1.014539,1.015337,1.011819,1.013560,1.012541,1.010561,1.014470,1.013863,1.013603,1.010509,...,1.010504,1.012292,1.014531,1.010875,1.013110,1.009393,1.011295,1.013535,1.012612,1.010975,1.008138,1.017008,1.011524,1.012853,1.010488,1.010685,1.009614,1.011463,1.010529,1.013514,1.013095,1.010180,1.009328,1.009959,1.011987,1.009067,1.010589,1.015265,1.011590,1.008476,1.007831,1.014301,1.011763,1.012854,1.010868,1.009067,1.013081,1.014052,1.014169,1.010700


In [45]:
conserv.tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
1195,1.008578,1.011381,1.014912,1.011712,1.011412,1.013448,1.012937,1.012713,1.012442,1.011877,1.010890,1.015085,1.011823,1.010774,1.013741,1.012463,1.010481,1.012042,1.010013,1.011650,1.010702,1.015177,1.009268,1.015802,1.013246,1.011480,1.014262,1.011808,1.013035,1.012188,1.011991,1.011983,1.013620,1.007192,1.011025,1.012090,1.008341,1.010277,1.008263,1.009864,...,1.013407,1.012236,1.008801,1.011556,1.014132,1.013167,1.011930,1.005728,1.012135,1.012189,1.008105,1.012535,1.008729,1.012677,1.008645,1.015812,1.012244,1.011646,1.011129,1.011375,1.011977,1.010045,1.011202,1.011145,1.012405,1.009502,1.010416,1.014555,1.010325,1.008455,1.011110,1.018551,1.017164,1.011033,1.011622,1.010637,1.015141,1.016908,1.012418,1.012384
1196,1.010156,1.011412,1.014954,1.013829,1.011331,1.011714,1.012388,1.015585,1.009626,1.010456,1.013386,1.011552,1.012214,1.009590,1.014809,1.017939,1.012175,1.015767,1.014235,1.010038,1.010174,1.009347,1.008282,1.008242,1.011491,1.010348,1.008671,1.012711,1.011370,1.011566,1.016173,1.014568,1.010675,1.011423,1.015864,1.011836,1.008628,1.015235,1.013844,1.010202,...,1.010162,1.012565,1.015078,1.008071,1.011979,1.011206,1.010430,1.012434,1.009285,1.011256,1.014472,1.008384,1.011580,1.013295,1.012103,1.008887,1.009869,1.012205,1.014248,1.012801,1.011151,1.012475,1.009491,1.009335,1.013698,1.013819,1.013943,1.013138,1.010620,1.010007,1.014089,1.005280,1.011019,1.014622,1.014707,1.011331,1.008959,1.011845,1.009231,1.013825
1197,1.009489,1.012945,1.010816,1.007569,1.012489,1.010699,1.013583,1.007723,1.006967,1.010850,1.011487,1.012201,1.011311,1.014177,1.011957,1.013377,1.013618,1.011167,1.011012,1.013602,1.008055,1.011420,1.013161,1.011338,1.009541,1.009457,1.010572,1.011389,1.008850,1.009914,1.009903,1.011550,1.009916,1.013541,1.014053,1.010536,1.012873,1.014833,1.008448,1.012945,...,1.008158,1.011632,1.013642,1.010293,1.011147,1.010385,1.010496,1.011645,1.012282,1.008499,1.007700,1.011233,1.009720,1.014477,1.008482,1.012891,1.012686,1.007112,1.007685,1.013563,1.012132,1.013158,1.007635,1.012636,1.015906,1.012135,1.013687,1.014504,1.009259,1.008393,1.012554,1.012438,1.010846,1.013605,1.010694,1.010242,1.010384,1.013582,1.009917,1.011624
1198,1.012788,1.009562,1.013002,1.013384,1.010478,1.016235,1.011515,1.011403,1.010571,1.011338,1.009562,1.010559,1.011693,1.010253,1.009893,1.015397,1.008411,1.009082,1.011324,1.014249,1.014912,1.010767,1.014639,1.011280,1.009437,1.013349,1.010671,1.015951,1.011889,1.011628,1.013140,1.014128,1.011160,1.011490,1.010150,1.005563,1.017855,1.011967,1.013426,1.015285,...,1.010314,1.012369,1.012387,1.010673,1.013239,1.008918,1.013827,1.011421,1.013611,1.009328,1.011560,1.011944,1.011847,1.007157,1.012671,1.012645,1.012776,1.011757,1.008390,1.014953,1.012512,1.010788,1.008020,1.012332,1.012794,1.010393,1.010366,1.013865,1.011880,1.008042,1.012182,1.012677,1.010571,1.008755,1.009961,1.015417,1.016127,1.009279,1.010671,1.011377
1199,1.010232,1.012780,1.010917,1.011199,1.010444,1.013054,1.016761,1.011381,1.012126,1.011173,1.009829,1.012615,1.011435,1.008715,1.010152,1.010429,1.008757,1.008378,1.011939,1.010601,1.013702,1.012241,1.009898,1.013519,1.007858,1.009548,1.008376,1.008446,1.013951,1.012316,1.015638,1.011539,1.014388,1.011561,1.012870,1.014101,1.006919,1.013320,1.010817,1.010271,...,1.012989,1.007689,1.011735,1.007859,1.009662,1.012935,1.013307,1.009699,1.012951,1.011353,1.014524,1.014426,1.015875,1.012239,1.011257,1.009897,1.009883,1.013512,1.010567,1.009734,1.015590,1.009276,1.010881,1.014966,1.010201,1.014179,1.016228,1.010696,1.012761,1.013623,1.014548,1.014911,1.009834,1.009143,1.015073,1.012140,1.016510,1.010047,1.009365,1.011254


In [46]:
conservative.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
0,101239.838644,101008.912977,101325.638535,101292.752970,101200.493974,101227.299451,100740.208869,100849.736469,101276.013168,101256.046311,101034.398512,101026.036525,101221.672720,101227.221817,101139.015437,101363.229715,101026.473083,101174.318359,101540.057790,101049.264308,101019.392374,101096.304588,101175.734630,101101.345678,100934.106896,101176.611211,101328.934388,101269.058873,101455.664124,101400.963786,101025.411300,101330.246281,101024.957679,101579.747495,101469.174521,100972.429903,101285.440184,101248.762628,101100.869778,100886.453413,...,101293.496167,100985.591827,101282.464981,101320.335306,101180.578226,101349.801463,101042.302755,101386.813012,101363.828688,100877.938952,101062.871624,101019.853530,101277.397803,101368.956154,101080.694414,101409.429436,101349.435082,101158.173298,100960.860054,101219.699830,101515.805227,101127.206765,101123.274139,100865.943975,101045.073688,101343.848789,101301.658983,101221.972497,101313.313500,101498.655586,101296.962352,101113.798489,101425.026865,101364.678023,100750.388512,100639.163518,101131.562348,101530.588780,100756.603957,100973.109176
1,101266.693303,101437.505840,100988.903696,101000.646809,101082.489212,101357.714767,101315.553131,101164.298655,100843.960091,101156.604353,100940.979754,101486.529503,100837.718769,100970.711121,101143.540755,100867.162660,100912.289694,101400.377015,101390.831315,101267.781852,101282.842711,101004.184306,100764.149717,100794.948715,101393.557469,100983.057999,101051.577095,101329.327182,101203.701120,100787.132569,101149.678616,101158.360885,101281.216589,100468.941263,101218.327247,101069.152579,101158.044526,100906.439055,101103.201992,101384.096478,...,100991.221369,100804.583397,101121.654811,100835.890146,101175.969158,100759.716638,100757.497886,101336.062685,101113.431381,101652.317473,101483.303303,101433.895531,100659.801646,101110.517600,101455.146682,100824.707538,101012.939226,101119.772276,101459.759971,100971.609581,101103.824667,101389.324506,101344.236206,100933.160932,100871.144399,101170.130560,101257.737911,100943.419619,101467.935728,101162.922382,101208.388990,101340.489585,101166.005478,101702.680665,101255.017173,100912.599970,101324.798868,101132.145722,101202.326838,101031.890911
2,100940.128985,101179.948735,100925.979440,101429.822478,101100.773023,100973.104449,101163.884176,100931.473742,100982.393459,101617.073325,101179.042752,100857.535429,100877.533033,100691.840628,101480.528893,101129.586521,101192.380631,100928.478594,101284.539530,101361.766773,101095.454510,101293.780044,101234.415762,101094.042922,101172.325292,100631.537992,101072.941525,101355.228704,100698.763839,100799.840649,100926.907457,101215.144112,101157.355647,100612.874630,101239.721274,101340.066537,101026.080590,101431.262633,101005.781318,101299.233671,...,101164.301321,100759.755607,101619.871842,101502.326430,101362.208813,101278.655417,101274.795672,100988.971072,101437.818692,100903.065444,101496.161540,100921.236548,100724.419912,101249.069245,101255.135040,101468.061015,101149.487540,101437.062842,101485.079814,100890.107971,101254.862039,101437.499451,101098.286195,100703.587033,101005.199303,101237.960149,101205.079518,101194.689909,101167.977765,100745.853416,101550.003837,101094.864977,101266.957980,100984.752381,101193.375704,101400.948962,101253.821250,100992.877340,101412.251144,100740.176396
3,100994.774691,100941.086185,101213.379525,101171.626304,101250.386507,100665.220059,100853.479974,101238.760474,100952.342203,101348.427378,101042.675281,101179.281991,100920.867541,101090.973856,101150.401411,101506.159149,101234.042918,101085.797361,101281.767157,101336.549153,101257.087542,101067.971889,101282.723002,100804.026314,101105.67

In [47]:
conservative.tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
1195,100857.811299,101138.089194,101491.165895,101171.232256,101141.158578,101344.799835,101293.716246,101271.340012,101244.208683,101187.742073,101089.025995,101508.508922,101182.321986,101077.440531,101374.070945,101246.259057,101048.061219,101204.249907,101001.324111,101165.019419,101070.202887,101517.651136,100926.785023,101580.178085,101324.579371,101148.001787,101426.240265,101180.838048,101303.493551,101218.825536,101199.058509,101198.341471,101361.995253,100719.178779,101102.515105,101208.997893,100834.084302,101027.655339,100826.317434,100986.401986,...,101340.663977,101223.553831,100880.123835,101155.616376,101413.214302,101316.699032,101193.036722,100572.772516,101213.464543,101218.905516,100810.478176,101253.524925,100872.931229,101267.656367,100864.541763,101581.249299,101224.380002,101164.640409,101112.945822,101137.471222,101197.708716,101004.473407,101120.212198,101114.511999,101240.524352,100950.220600,101041.577265,101455.484330,101032.461875,100845.482227,101111.041328,101855.125166,101716.406847,101103.257463,101162.215904,101063.671417,101514.059785,101690.842661,101241.794575,101238.422961
1196,101015.634791,101141.240501,101495.406589,101382.945056,101133.140927,101171.407706,101238.757390,101558.452330,100962.618929,101045.580830,101338.591839,101155.235616,101221.400378,100958.957139,101480.852087,101793.900615,101217.491054,101576.686379,101423.545077,101003.815457,101017.356982,100934.660745,100828.163455,100824.151029,101149.116041,101034.762331,100867.116803,101271.063147,101137.017507,101156.588397,101617.329825,101456.770838,101067.525628,101142.343247,101586.387716,101183.623887,100862.831020,101523.469743,101384.447950,101020.222110,...,101016.233555,101256.456473,101507.839523,100807.087774,101197.896392,101120.607311,101043.022544,101243.373340,100928.488152,101125.565712,101447.222424,100838.395633,101158.014252,101329.536607,101210.324605,100888.708911,100986.871360,101220.471094,101424.799021,101280.051961,101115.075122,101247.549261,100949.116011,100933.452014,101369.792703,101381.902253,101394.339136,101313.800279,101062.039071,101000.682332,101408.853729,100528.018836,101101.933746,101462.231100,101470.683618,101133.125257,100895.909303,101184.490846,100923.105017,101382.536169
1197,100948.945373,101294.485024,101081.614689,100756.914053,101248.909978,101069.857355,101358.290662,100772.334517,100696.685575,101084.974966,101148.671703,101220.132775,101131.071853,101417.675121,101195.663193,101337.707349,101361.790610,101116.658661,101101.197331,101360.249159,100805.461505,101141.980728,101316.135290,101133.815801,100954.125975,100945.710089,101057.225891,101138.881982,100884.995578,100991.368972,100990.273928,101155.032564,100991.577857,101354.108868,101405.315685,101053.579828,101287.296281,101483.270125,100844.777228,101294.514456,...,100815.797405,101163.154224,101364.169347,101029.310898,101114.749812,101038.518114,101049.621629,101164.485694,101228.176263,100849.862507,100770.006946,101123.256097,100971.988216,101447.662091,100848.213807,101289.107690,101268.592985,100711.230768,100768.507378,101356.287931,101213.241226,101315.823240,100763.540122,101263.585460,101590.632187,101213.464160,101368.721809,101450.406480,100925.922076,100839.251793,101255.449838,101243.844293,101084.639608,101360.502763,101069.397385,101024.164202,101038.377519,101358.239870,100991.747522,101162.360653
1198,101278.756130,100956.228410,101300.159677,101338.394025,101047.797548,101623.526496,101151.495588,101140.324056,101057.084229,101133.777239,100956.153013,101055.870963,101169.348235,101025.262399,100989.316442,101539.695258,100841.138014,100908.206168,101132.350549,101424.947629,101491.160896,101076.737804,101463.944924,101128.0442

###### *Cautious Portfolio Simulation*

In [48]:
# Calculates portfolio expected returns from the portfolio ratios and the predicted returns 

stb_b = stb_abs_pred * stb_abs2
stb_g = stb_gtb_pred * stb_GI
axa_b = axa_mmf_pred * axa_mmf2

# 100000 is the invested amount
caut = stb_b.add(stb_g.add(axa_b))
cautious = caut * 100000

#cautious.insert(loc = index, column = 'Date', value = future_dates.strftime('%Y-%m'))

caut.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
0,1.012151,1.009725,1.013411,1.013579,1.012799,1.012152,1.007672,1.009520,1.012454,1.012570,1.010520,1.010849,1.012689,1.012837,1.011581,1.013810,1.010795,1.011232,1.014495,1.011612,1.010277,1.011505,1.011313,1.011159,1.009196,1.011506,1.013634,1.013704,1.014354,1.013879,1.011210,1.012854,1.010480,1.015869,1.013810,1.010308,1.013174,1.012759,1.010984,1.010316,...,1.012888,1.009996,1.012139,1.013712,1.012242,1.013062,1.010113,1.014527,1.014048,1.009339,1.011828,1.010590,1.013022,1.013595,1.011172,1.013397,1.012743,1.012566,1.009452,1.011404,1.014144,1.011774,1.011655,1.009169,1.010493,1.013306,1.012748,1.012443,1.013049,1.015060,1.012813,1.011066,1.014248,1.013444,1.008739,1.007718,1.010911,1.014767,1.007781,1.010521
1,1.012829,1.014502,1.010271,1.010419,1.009779,1.013134,1.012365,1.010737,1.009112,1.011439,1.010510,1.013577,1.009873,1.010045,1.012016,1.008985,1.009773,1.013219,1.013650,1.012848,1.012264,1.010374,1.008082,1.009471,1.014355,1.010046,1.010647,1.013044,1.012332,1.008678,1.012870,1.011942,1.012670,1.006744,1.012102,1.011161,1.011154,1.009624,1.011433,1.013835,...,1.010068,1.009092,1.011335,1.009037,1.011908,1.008178,1.008588,1.013701,1.011143,1.015739,1.013548,1.013161,1.007125,1.011752,1.013538,1.008991,1.010394,1.011377,1.014054,1.009918,1.011848,1.013146,1.012869,1.010174,1.008775,1.011390,1.012569,1.009667,1.013754,1.012677,1.011506,1.013356,1.012602,1.015806,1.013057,1.010708,1.013205,1.011070,1.012107,1.011158
2,1.009902,1.011946,1.009694,1.013873,1.010950,1.011345,1.011084,1.010439,1.010940,1.014990,1.012447,1.009059,1.010073,1.007605,1.014204,1.011997,1.011512,1.009933,1.012614,1.013707,1.011139,1.012517,1.011800,1.011054,1.012600,1.007814,1.010374,1.013402,1.008135,1.008397,1.009307,1.013334,1.011734,1.006335,1.012152,1.012871,1.009950,1.013955,1.010695,1.013004,...,1.012231,1.008004,1.016004,1.014249,1.013281,1.012610,1.013363,1.010842,1.014057,1.010259,1.015900,1.009810,1.008419,1.012215,1.013032,1.014063,1.011946,1.013651,1.014688,1.009547,1.012401,1.014215,1.012154,1.008268,1.010467,1.013171,1.012059,1.011565,1.011430,1.008802,1.015222,1.011898,1.012023,1.010577,1.012163,1.015003,1.013194,1.010240,1.013179,1.008929
3,1.009872,1.009904,1.012428,1.012899,1.011887,1.007283,1.009279,1.012950,1.010039,1.013258,1.011157,1.011471,1.009834,1.010630,1.011535,1.015098,1.012617,1.011561,1.012724,1.013813,1.012793,1.011460,1.013249,1.009852,1.011162,1.012503,1.012845,1.013823,1.011316,1.017027,1.009738,1.009872,1.013937,1.010537,1.013294,1.010213,1.014755,1.009218,1.010666,1.010985,...,1.012640,1.013377,1.012023,1.013706,1.010641,1.012501,1.010193,1.013380,1.009899,1.012086,1.012191,1.013634,1.012316,1.011099,1.011656,1.008792,1.012907,1.013526,1.014228,1.013344,1.011632,1.013296,1.011158,1.009525,1.011580,1.013070,1.011609,1.011449,1.009059,1.012918,1.011817,1.014051,1.009858,1.011547,1.011571,1.014242,1.013628,1.010948,1.012401,1.010828
4,1.011739,1.010729,1.013087,1.006565,1.015227,1.009466,1.010667,1.013589,1.013923,1.012896,1.010422,1.010859,1.007131,1.010164,1.008976,1.011173,1.012440,1.013100,1.010227,1.011723,1.011669,1.010645,1.012027,1.012016,1.009179,1.009664,1.011592,1.012976,1.012615,1.011228,1.014233,1.014483,1.011530,1.013763,1.012587,1.010895,1.014440,1.014264,1.013154,1.011909,...,1.011419,1.011618,1.013628,1.011377,1.013463,1.010147,1.012821,1.012817,1.012332,1.011198,1.009433,1.017977,1.011910,1.012462,1.010638,1.010636,1.009152,1.011358,1.011158,1.013356,1.013343,1.010293,1.010663,1.010373,1.013324,1.009502,1.010728,1.014683,1.012542,1.009421,1.008765,1.013466,1.011480,1.012898,1.013017,1.010294,1.012770,1.013710,1.013400,1.009820


In [49]:
caut.tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
1195,1.010169,1.011589,1.015205,1.011448,1.011107,1.013189,1.012617,1.012275,1.012187,1.012046,1.010650,1.014431,1.012138,1.010965,1.013646,1.012190,1.011126,1.012316,1.010000,1.011445,1.010925,1.015958,1.009996,1.016306,1.012446,1.012124,1.013924,1.011783,1.013282,1.012481,1.013149,1.012601,1.013492,1.008108,1.012370,1.011323,1.009770,1.011018,1.008541,1.010468,...,1.013299,1.012063,1.009545,1.011641,1.014557,1.013815,1.010915,1.006185,1.011604,1.012614,1.008727,1.011829,1.009552,1.012331,1.009117,1.015176,1.011082,1.011656,1.010912,1.010319,1.012249,1.010784,1.011863,1.011534,1.012678,1.009358,1.011218,1.014549,1.010284,1.008521,1.010746,1.018607,1.016253,1.011514,1.011587,1.010141,1.014820,1.016094,1.012843,1.012810
1196,1.010910,1.012742,1.015034,1.012843,1.011040,1.011786,1.012581,1.015174,1.011233,1.010022,1.013101,1.011589,1.012466,1.010205,1.014730,1.017639,1.010954,1.015624,1.013255,1.009931,1.011284,1.010034,1.009961,1.009300,1.011299,1.010677,1.009507,1.012567,1.011936,1.011065,1.015474,1.015235,1.011355,1.011919,1.015027,1.011637,1.009168,1.015139,1.013573,1.011520,...,1.009242,1.012785,1.014926,1.009287,1.011903,1.010749,1.011923,1.012604,1.010136,1.012048,1.013939,1.009358,1.011907,1.013035,1.011735,1.009768,1.010313,1.012683,1.013638,1.012087,1.011901,1.011593,1.010027,1.009391,1.014272,1.014016,1.014020,1.013224,1.010800,1.010369,1.013607,1.005914,1.011063,1.014184,1.013630,1.010954,1.010312,1.010805,1.009405,1.012569
1197,1.010602,1.013282,1.011579,1.008768,1.012923,1.010232,1.012769,1.008688,1.008124,1.011220,1.012225,1.012662,1.011999,1.013896,1.011584,1.013599,1.014076,1.010619,1.011421,1.013532,1.008853,1.011994,1.012986,1.011468,1.010098,1.010185,1.010975,1.011684,1.009282,1.010103,1.010041,1.011674,1.010502,1.013434,1.012907,1.011113,1.012921,1.014869,1.008585,1.012914,...,1.008535,1.011955,1.012675,1.009657,1.011732,1.010182,1.010532,1.011308,1.011964,1.008287,1.007933,1.011543,1.010350,1.014050,1.008534,1.013339,1.012589,1.008150,1.008494,1.013024,1.012060,1.012983,1.008144,1.012864,1.014850,1.011984,1.014780,1.014472,1.009196,1.009579,1.012725,1.012246,1.010768,1.013662,1.010087,1.010034,1.010386,1.013117,1.009642,1.011343
1198,1.012944,1.010003,1.013725,1.013948,1.010191,1.016055,1.011971,1.011097,1.010755,1.011367,1.009924,1.010460,1.012406,1.010830,1.010353,1.013865,1.009476,1.010292,1.011727,1.014813,1.013744,1.010834,1.013736,1.011245,1.010775,1.012960,1.010761,1.014751,1.012106,1.012020,1.013101,1.014803,1.010960,1.012087,1.010231,1.006861,1.016766,1.011376,1.014180,1.014870,...,1.011016,1.013167,1.012369,1.011835,1.012399,1.009584,1.014795,1.011806,1.013273,1.010168,1.011607,1.011880,1.012326,1.007526,1.013523,1.012274,1.013200,1.011724,1.008718,1.014640,1.012351,1.011027,1.007532,1.012098,1.013144,1.010162,1.011139,1.013792,1.011786,1.009168,1.012396,1.012272,1.011755,1.009323,1.010960,1.015152,1.014957,1.009984,1.010552,1.010885
1199,1.010967,1.012094,1.011227,1.011903,1.010654,1.012022,1.016962,1.011379,1.011974,1.011874,1.009878,1.013140,1.011925,1.008996,1.010937,1.009953,1.010380,1.008891,1.011516,1.010919,1.012867,1.012976,1.010503,1.013990,1.009537,1.009254,1.009089,1.009207,1.013552,1.011883,1.014587,1.012606,1.013687,1.011380,1.012952,1.014653,1.008822,1.012788,1.011240,1.009735,...,1.012825,1.008842,1.011696,1.009156,1.009654,1.012260,1.013570,1.010263,1.014333,1.011827,1.013996,1.013716,1.015015,1.012475,1.011984,1.010251,1.010397,1.012674,1.010083,1.009758,1.015167,1.010196,1.010153,1.014155,1.010064,1.013865,1.015065,1.010832,1.012134,1.013775,1.014618,1.015232,1.009848,1.009831,1.014651,1.011261,1.014925,1.010304,1.009850,1.010990


In [50]:
cautious.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
0,101215.076127,100972.462366,101341.087921,101357.870507,101279.885147,101215.180865,100767.168078,100951.992142,101245.367114,101257.028169,101051.999908,101084.932948,101268.874530,101283.665719,101158.121693,101380.953925,101079.525734,101123.236602,101449.531952,101161.207670,101027.683149,101150.516770,101131.250724,101115.889843,100919.559292,101150.627783,101363.369722,101370.399125,101435.445096,101387.931305,101121.013306,101285.423206,101047.953194,101586.875521,101380.999071,101030.836270,101317.430152,101275.891716,101098.364838,101031.566602,...,101288.769471,100999.649301,101213.902625,101371.233963,101224.151309,101306.199084,101011.291708,101452.729059,101404.761492,100933.861839,101182.770996,101059.045039,101302.240523,101359.545080,101117.233784,101339.724267,101274.319194,101256.618762,100945.168305,101140.414729,101414.400799,101177.428107,101165.492466,100916.912589,101049.257476,101330.613149,101274.805307,101244.276210,101304.877866,101505.981689,101281.293847,101106.641545,101424.765047,101344.410622,100873.930940,100771.804842,101091.082717,101476.734435,100778.147125,101052.105322
1,101282.876268,101450.219914,101027.090394,101041.915672,100977.940756,101313.364529,101236.521130,101073.686787,100911.217504,101143.930089,101051.034363,101357.653926,100987.312231,101004.481389,101201.611004,100898.497785,100977.343724,101321.857679,101364.957759,101284.834980,101226.353767,101037.367886,100808.183090,100947.144132,101435.451704,101004.629617,101064.689199,101304.360741,101233.189480,100867.844232,101287.016132,101194.150759,101267.012808,100674.366779,101210.211755,101116.092324,101115.359286,100962.405879,101143.259297,101383.519028,...,101006.833229,100909.164249,101133.457856,100903.656452,101190.834463,100817.827430,100858.823649,101370.095135,101114.272399,101573.929887,101354.803384,101316.090284,100712.483868,101175.230265,101353.834523,100899.057718,101039.360956,101137.725673,101405.361722,100991.768549,101184.804415,101314.611696,101286.858684,101017.407890,100877.514646,101138.957964,101256.868738,100966.748278,101375.437440,101267.721125,101150.578683,101335.598287,101260.248230,101580.645904,101305.696874,101070.808772,101320.497389,101107.028617,101210.738676,101115.839738
2,100990.208676,101194.638796,100969.426969,101387.263303,101095.021140,101134.499004,101108.401260,101043.908463,101093.961506,101499.036883,101244.715436,100905.948372,101007.269603,100760.484003,101420.372876,101199.731422,101151.169498,100993.320467,101261.425217,101370.708718,101113.942137,101251.717316,101179.957997,101105.372292,101260.032253,100781.448649,101037.441079,101340.218757,100813.533729,100839.737237,100930.720584,101333.408185,101173.391754,100633.476332,101215.227991,101287.104281,100994.986556,101395.504689,101069.510552,101300.428930,...,101223.106014,100800.427074,101600.403985,101424.853907,101328.054878,101261.039738,101336.259868,101084.176943,101405.667904,101025.941563,101590.002035,100980.958275,100841.850711,101221.499195,101303.190628,101406.342148,101194.619369,101365.096163,101468.806895,100954.703681,101240.125511,101421.481318,101215.400976,100826.804128,101046.654188,101317.137023,101205.861421,101156.522137,101142.995585,100880.236450,101522.194988,101189.844054,101202.306149,101057.729302,101216.323677,101500.281263,101319.411470,101023.972586,101317.856937,100892.867586
3,100987.153876,100990.384250,101242.835324,101289.867764,101188.732507,100728.295048,100927.944190,101295.032832,101003.939453,101325.763176,101115.709732,101147.072844,100983.385464,101062.976371,101153.504618,101509.805520,101261.687422,101156.121839,101272.382194,101381.303755,101279.320406,101145.977171,101324.947939,100985.215235,101116.21

In [51]:
cautious.tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
1195,101016.866226,101158.928875,101520.476544,101144.838932,101110.696517,101318.910316,101261.671301,101227.482094,101218.698479,101204.628247,101065.047269,101443.088528,101213.812900,101096.539512,101364.556883,101218.993399,101112.639732,101231.607755,101000.009423,101144.500319,101092.512687,101595.848707,100999.570991,101630.567422,101244.649875,101212.391246,101392.434918,101178.309954,101328.237442,101248.085998,101314.901929,101260.117676,101349.161792,100810.791420,101236.952584,101132.277444,100977.002311,101101.771470,100854.071833,101046.759219,...,101329.944960,101206.344687,100954.465184,101164.102937,101455.697300,101381.459256,101091.486133,100618.450581,101160.382793,101261.394029,100872.666091,101182.945570,100955.166683,101233.099068,100911.748480,101517.582595,101108.220198,101165.570576,101091.168621,101031.949567,101224.867886,101078.440259,101186.272872,101153.351991,101267.849051,100935.835702,101121.763555,101454.879756,101028.402082,100852.103589,101074.570300,101860.671682,101625.336336,101151.421013,101158.706242,101014.068361,101481.970968,101609.439236,101284.266477,101280.998500
1196,101091.016903,101274.205419,101503.413016,101284.323563,101104.043039,101178.646802,101258.102247,101517.360056,101123.330979,101002.225589,101310.053451,101158.857821,101246.603355,101020.482310,101473.007923,101763.897964,101095.439401,101562.408784,101325.519340,100993.097896,101128.412274,101003.386438,100996.146687,100929.979885,101129.943113,101067.702945,100950.741630,101256.749500,101193.638007,101106.470924,101547.360516,101523.542856,101135.536573,101191.907556,101502.681761,101163.689303,100916.842847,101513.922874,101357.299969,101151.996380,...,100924.175197,101278.485704,101492.643188,100928.715373,101190.286762,101074.927179,101192.291565,101260.419270,101013.614995,101204.806451,101393.875152,100935.764474,101190.674604,101303.455072,101173.471079,100976.769314,101031.299944,101268.337971,101363.826489,101208.698096,101190.096828,101159.272190,101002.748514,100939.056141,101427.245961,101401.556170,101402.035482,101322.435373,101080.022212,101036.860408,101360.651749,100591.437460,101106.338259,101418.392777,101362.985628,101095.403211,101031.243069,101080.505799,100940.514846,101256.871798
1197,101060.200279,101328.184934,101157.911181,100876.810302,101292.278484,101023.176736,101276.947407,100868.829979,100812.367354,101122.034447,101222.511454,101266.208890,101199.930926,101389.619359,101158.429766,101359.888700,101407.603442,101061.868141,101142.112492,101353.209240,100885.299913,101199.418980,101298.624249,101146.754165,101009.793128,101018.505601,101097.505407,101168.447795,100928.154869,101010.282494,101004.096020,101167.429761,101050.185839,101343.412292,101290.739818,101111.340082,101292.089185,101486.875083,100858.485555,101291.400156,...,100853.504964,101195.494197,101267.451071,100965.721339,101173.229110,101018.211835,101053.248929,101130.779885,101196.423823,100828.726453,100793.312740,101154.254054,101034.985307,101404.978295,100853.357720,101333.894462,101258.888214,100814.958599,100849.432463,101302.358985,101205.989395,101298.342430,100814.429723,101286.424387,101484.989735,101198.381524,101478.004636,101447.228073,100919.566505,100957.904147,101272.499791,101224.561924,101076.784013,101366.160559,101008.660209,101003.445204,101038.640499,101311.667813,100964.188961,101134.311192
1198,101294.438867,101000.257514,101372.510159,101394.824659,101019.116191,101605.516734,101197.065723,101109.719252,101075.518096,101136.694852,100992.431519,101046.028543,101240.626802,101082.956275,101035.268443,101386.492440,100947.557139,101029.242853,101172.710598,101481.343446,101374.406454,101083.404973,101373.603470,101124.4513

###### *Balanced Portfolio Simulation*

In [52]:
# Calculates portfolio expected returns from the portfolio ratios and the predicted returns 

stb_c = stb_abs_pred * stb_abs3
axa_c = axa_mmf_pred * axa_mmf3

# 100000 is the invested amount
bal = stb_c.add(axa_c)
balanced = bal * 100000

#balanced.insert(loc = index, column = 'Date', value = future_dates.strftime('%Y-%m'))

bal.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
0,1.010270,1.008011,1.013193,1.016230,1.014249,1.012029,1.008535,1.013277,1.011173,1.012067,1.010103,1.012608,1.015340,1.014201,1.012318,1.015574,1.012275,1.008988,1.011851,1.014813,1.009718,1.014829,1.011217,1.011328,1.008889,1.009774,1.014534,1.016312,1.013890,1.013076,1.012304,1.012285,1.011664,1.014553,1.012251,1.012426,1.014009,1.013744,1.011283,1.017408,...,1.010985,1.010158,1.009066,1.014937,1.015082,1.009806,1.009202,1.016622,1.015013,1.010260,1.016421,1.013308,1.014423,1.012371,1.011382,1.008988,1.009432,1.016830,1.009820,1.008402,1.011053,1.014473,1.013158,1.010300,1.010753,1.013152,1.011569,1.011766,1.013026,1.017107,1.013413,1.009940,1.014957,1.012387,1.011795,1.012738,1.009896,1.012701,1.009116,1.014392
1,1.014526,1.013160,1.010498,1.012188,1.006105,1.011565,1.009539,1.009627,1.010568,1.010757,1.014599,1.008000,1.014552,1.010659,1.013497,1.010951,1.013709,1.010817,1.013306,1.013756,1.011455,1.011157,1.010332,1.014777,1.014889,1.010093,1.010117,1.013080,1.012416,1.012761,1.019155,1.013921,1.012737,1.013721,1.012820,1.013119,1.009410,1.012861,1.011630,1.013734,...,1.010684,1.011675,1.010417,1.009984,1.012745,1.010428,1.010762,1.015073,1.010490,1.012220,1.008710,1.009185,1.009024,1.013999,1.011103,1.012594,1.010714,1.012930,1.012297,1.010044,1.015325,1.014038,1.013214,1.013778,1.008403,1.009632,1.014683,1.010211,1.008886,1.016516,1.010324,1.010192,1.016733,1.012595,1.014550,1.016726,1.012575,1.009730,1.010690,1.014532
2,1.013220,1.012323,1.012815,1.013545,1.010859,1.015082,1.008648,1.013997,1.012787,1.010606,1.014692,1.011365,1.014843,1.009500,1.013367,1.014427,1.009561,1.014828,1.012621,1.014022,1.010942,1.013778,1.008612,1.013009,1.016243,1.010880,1.012146,1.012929,1.010660,1.009158,1.009643,1.019459,1.012416,1.008059,1.010013,1.011362,1.008312,1.012253,1.012141,1.014717,...,1.014626,1.011817,1.017799,1.015990,1.010890,1.012427,1.016335,1.014899,1.011844,1.012825,1.018164,1.010601,1.012174,1.012792,1.014185,1.011372,1.010370,1.009871,1.014011,1.011063,1.011506,1.013762,1.017844,1.012285,1.011736,1.015414,1.013586,1.010338,1.010402,1.013176,1.016177,1.014365,1.010332,1.014152,1.013348,1.019050,1.015633,1.012763,1.011071,1.014404
3,1.008593,1.010442,1.014534,1.015592,1.012509,1.010253,1.012191,1.013594,1.011489,1.013297,1.015106,1.009363,1.011980,1.007135,1.012599,1.015040,1.015102,1.014230,1.011698,1.015031,1.013209,1.014540,1.014474,1.016429,1.010750,1.012398,1.012879,1.008391,1.009658,1.014234,1.012621,1.007353,1.014735,1.014121,1.014583,1.016311,1.013759,1.011773,1.012365,1.015270,...,1.008949,1.009181,1.011732,1.007644,1.011932,1.011041,1.013546,1.014173,1.012582,1.014326,1.010959,1.015002,1.011917,1.010056,1.010445,1.010789,1.014589,1.011483,1.012555,1.012417,1.014948,1.014747,1.013325,1.009322,1.010890,1.013133,1.011178,1.012915,1.008229,1.015238,1.017689,1.016718,1.006977,1.012227,1.007450,1.012640,1.014599,1.007568,1.013351,1.011936
4,1.012595,1.011631,1.015474,1.007893,1.017450,1.008560,1.014943,1.015666,1.015316,1.012257,1.009239,1.015033,1.009650,1.008580,1.015993,1.011784,1.013344,1.017453,1.015826,1.012952,1.011107,1.014827,1.013858,1.013634,1.005100,1.012565,1.015826,1.012907,1.012084,1.010971,1.012913,1.011846,1.009163,1.012026,1.011870,1.012874,1.015783,1.016455,1.013323,1.017619,...,1.015030,1.009996,1.010434,1.015414,1.015746,1.011318,1.016903,1.011502,1.011182,1.010630,1.012464,1.019921,1.014107,1.013235,1.009813,1.011135,1.009612,1.012076,1.014589,1.012658,1.014205,1.011047,1.015051,1.011438,1.016008,1.010395,1.009424,1.013921,1.013721,1.012327,1.008591,1.010187,1.011441,1.013609,1.021086,1.013433,1.011564,1.011532,1.011443,1.007381


In [53]:
bal.tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
1195,1.015002,1.013924,1.015838,1.011779,1.012317,1.011843,1.011447,1.011915,1.013115,1.012683,1.010122,1.013621,1.013764,1.011948,1.013566,1.010411,1.013232,1.016513,1.009864,1.009753,1.012420,1.017926,1.013009,1.017886,1.010742,1.013691,1.011160,1.012095,1.014886,1.013207,1.015352,1.014643,1.012655,1.011435,1.018602,1.007639,1.012159,1.016585,1.010582,1.011263,...,1.013169,1.011528,1.013347,1.013894,1.015989,1.016460,1.008128,1.007414,1.008685,1.015189,1.008400,1.010044,1.012059,1.011117,1.012989,1.013069,1.006619,1.010812,1.009907,1.005940,1.014970,1.013614,1.013821,1.012827,1.013101,1.010024,1.015034,1.015939,1.007146,1.009748,1.010189,1.018697,1.015143,1.013233,1.012295,1.010273,1.012154,1.014897,1.014569,1.014229
1196,1.014433,1.016011,1.015474,1.010691,1.007786,1.012495,1.015479,1.013459,1.017466,1.008887,1.013391,1.011605,1.013760,1.009472,1.013283,1.017270,1.007844,1.015104,1.009805,1.009042,1.013321,1.011576,1.016507,1.013123,1.011641,1.012652,1.010827,1.012998,1.014914,1.009577,1.014414,1.019792,1.012785,1.014234,1.013314,1.012493,1.012027,1.014710,1.015106,1.014892,...,1.007595,1.012882,1.014233,1.012520,1.012837,1.009908,1.016363,1.015267,1.012709,1.013957,1.011060,1.012308,1.013299,1.011503,1.011665,1.012308,1.011325,1.015281,1.013770,1.009238,1.016044,1.010492,1.012425,1.008247,1.017870,1.013843,1.014907,1.015037,1.012805,1.011189,1.011896,1.008843,1.012442,1.014382,1.009613,1.009426,1.013330,1.006425,1.009689,1.009282
1197,1.014248,1.016076,1.015195,1.013153,1.015558,1.009232,1.008126,1.011368,1.012946,1.013446,1.015428,1.013027,1.014834,1.012836,1.010193,1.015729,1.016685,1.008512,1.012421,1.012887,1.009903,1.013007,1.013462,1.011679,1.009917,1.013256,1.010899,1.011693,1.011148,1.012632,1.010624,1.011333,1.013406,1.014525,1.006373,1.013104,1.013725,1.014521,1.008118,1.013206,...,1.011307,1.014978,1.009617,1.008974,1.014422,1.009426,1.013050,1.009934,1.010323,1.006690,1.008163,1.012609,1.012428,1.012799,1.008925,1.016169,1.011545,1.012096,1.011649,1.011164,1.011950,1.012755,1.010293,1.013986,1.013865,1.011873,1.018656,1.016183,1.009257,1.012592,1.012692,1.011921,1.012034,1.014026,1.009711,1.010421,1.010095,1.010932,1.009084,1.012182
1198,1.014115,1.012320,1.016809,1.015034,1.009818,1.017183,1.013951,1.010250,1.012932,1.011401,1.010305,1.010860,1.016302,1.013145,1.012262,1.009980,1.013903,1.013417,1.014925,1.014198,1.011133,1.011368,1.011065,1.011933,1.016511,1.013915,1.011808,1.012058,1.010405,1.012820,1.012305,1.015370,1.010652,1.014372,1.011340,1.010238,1.014172,1.010345,1.014444,1.014620,...,1.011764,1.015772,1.012872,1.014663,1.010328,1.011240,1.019361,1.012567,1.012796,1.011347,1.012233,1.013681,1.014542,1.008440,1.016496,1.011940,1.015513,1.011506,1.009395,1.014149,1.012198,1.012297,1.007088,1.010918,1.015016,1.008082,1.015558,1.012810,1.012175,1.013047,1.013548,1.008759,1.016963,1.011091,1.015365,1.016040,1.012580,1.011805,1.011833,1.012208
1199,1.012682,1.010606,1.013208,1.011219,1.012146,1.010878,1.019200,1.011800,1.012520,1.012538,1.009943,1.012876,1.011708,1.009549,1.012450,1.009203,1.014363,1.009670,1.010057,1.012630,1.011144,1.014798,1.012871,1.016849,1.014706,1.007323,1.011700,1.009769,1.016856,1.013236,1.011268,1.015766,1.012942,1.011942,1.013612,1.016183,1.016258,1.011006,1.013351,1.007539,...,1.013272,1.012965,1.013663,1.014724,1.010621,1.010720,1.015552,1.011214,1.017426,1.010820,1.013653,1.012301,1.012003,1.014413,1.012616,1.009950,1.011698,1.009563,1.008449,1.008345,1.013545,1.012365,1.007762,1.012521,1.007841,1.013832,1.011322,1.013019,1.011593,1.014620,1.014661,1.015512,1.009004,1.013917,1.015960,1.009555,1.008762,1.011119,1.010268,1.009971


In [54]:
balanced.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
0,101027.003312,100801.137460,101319.298379,101623.032966,101424.897070,101202.879309,100853.518202,101327.732846,101117.314469,101206.678530,101010.253806,101260.829150,101534.011337,101420.132923,101231.803802,101557.383810,101227.517254,100898.774863,101185.055609,101481.290048,100971.771756,101482.931037,101121.709546,101132.784950,100888.876925,100977.428918,101453.432044,101631.239590,101388.998199,101307.635816,101230.403046,101228.545922,101166.441169,101455.317553,101225.058873,101242.611806,101400.927905,101374.352917,101128.329414,101740.839941,...,101098.539807,101015.782546,100906.645623,101493.722269,101508.207487,100980.621818,100920.247029,101662.248101,101501.292142,101025.974583,101642.125725,101330.814040,101442.280927,101237.142556,101138.180818,100898.797089,100943.229481,101683.045654,100982.040345,100840.170138,101105.277520,101447.288958,101315.802741,101030.018281,101075.299262,101315.249979,101156.933286,101176.573471,101302.618696,101710.693274,101341.255382,100993.963591,101495.708974,101238.654320,101179.533014,101273.826474,100989.557065,101270.117174,100911.642886,101439.174788
1,101452.592939,101315.950115,101049.811523,101218.815191,100610.541564,101156.546628,100953.870933,100962.670468,101056.809367,101075.724796,101459.916698,100800.022370,101455.235875,101065.889548,101349.689844,101095.098155,101370.921333,101081.722965,101330.550092,101375.619114,101145.472970,101115.697785,101033.245933,101477.717067,101488.869638,101009.320307,101011.692839,101307.966805,101241.562031,101276.051700,101915.509298,101392.111072,101273.705719,101372.059925,101281.969150,101311.857074,100941.002099,101286.084122,101162.961815,101373.412141,...,101068.431112,101167.474796,101041.747424,100998.437392,101274.490780,101042.837045,101076.218548,101507.347155,101048.990292,101221.980825,100871.048858,100918.529137,100902.403311,101399.870195,101110.316958,101259.403010,101071.444170,101293.006030,101229.719092,101004.430081,101532.487924,101403.787737,101321.420578,101377.768942,100840.286027,100963.180111,101468.265223,101021.133142,100888.564655,101651.551203,101032.354925,101019.233685,101673.297726,101259.460651,101454.976857,101672.601375,101257.527778,100972.984782,101069.020946,101453.163166
2,101321.983994,101232.276761,101281.501800,101354.463822,101085.907344,101508.176419,100864.771315,101399.677518,101278.731554,101060.552645,101469.195526,101136.515497,101484.277053,100949.985695,101336.662051,101442.709617,100956.134688,101482.845834,101262.084459,101402.228066,101094.242790,101377.797035,100861.215494,101300.931359,101624.260626,101087.966164,101214.574193,101292.885508,101065.954848,100915.808172,100964.284802,101945.854815,101241.609760,100805.893915,101001.266336,101136.248243,100831.188242,101225.304627,101214.111261,101471.694040,...,101462.563679,101181.702114,101779.855996,101598.979546,101089.044343,101242.701757,101633.509882,101489.925254,101184.428756,101282.495817,101816.404113,101060.107708,101217.377407,101279.223263,101418.491371,101137.231390,101036.991074,100987.063562,101401.110901,101106.347499,101150.563414,101376.158695,101784.405322,101228.455298,101173.603381,101541.382138,101358.608460,101033.764224,101040.232612,101317.550266,101617.680934,101436.474554,101033.227913,101415.152713,101334.795878,101905.048073,101563.274216,101276.310313,101107.140061,101440.391576
3,100859.273070,101044.229043,101453.359053,101559.229592,101250.855547,101025.302142,101219.062864,101359.383978,101148.867418,101329.747741,101510.592958,100936.256834,101197.981813,100713.459272,101259.892335,101503.951639,101510.166346,101422.993862,101169.796336,101503.091787,101320.949995,101453.968684,101447.359434,101642.851106,101075.03

In [55]:
balanced.tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
1195,101500.173992,101392.420044,101583.808373,101177.871712,101231.713446,101184.262261,101144.695555,101191.488129,101311.532559,101268.273269,101012.205178,101362.109206,101376.367732,101194.783959,101356.644984,101041.124285,101323.194599,101651.333043,100986.387002,100975.253613,101242.039165,101792.561951,101300.879302,101788.593984,101074.247543,101369.058064,101116.017785,101209.530058,101488.610286,101320.667634,101535.209575,101464.312831,101265.464417,101143.522057,101860.161302,100763.855684,101215.910083,101658.492529,101058.203207,101126.250442,...,101316.863650,101152.839016,101334.732594,101389.429379,101598.857023,101645.955722,100812.775207,100741.409605,100868.512606,101518.867202,100840.041859,101004.363075,101205.874287,101111.697706,101298.861258,101306.871739,100661.929318,101081.221238,100990.727096,100594.008741,101497.013319,101361.387804,101382.054309,101282.719314,101310.092415,101002.362157,101503.420133,101593.929493,100714.636673,100974.781733,101018.863103,101869.706131,101514.281008,101323.343597,101229.513190,101027.345761,101215.397378,101489.744987,101456.892347,101422.911121
1196,101443.299676,101601.097881,101547.438303,101069.139658,100778.637552,101249.534038,101547.882456,101345.878588,101746.615535,100888.688115,101339.134198,101160.488677,101375.989398,100947.230586,101328.265868,101726.952442,100784.406814,101510.421429,100980.480463,100904.232506,101332.095363,101157.643247,101650.701434,101312.257944,101164.118158,101265.151389,101082.679645,101299.848021,101491.397607,100957.650016,101441.415883,101979.175108,101278.459350,101423.449102,101331.387619,101249.334395,101202.664939,101471.046845,101510.585159,101489.174325,...,100759.496797,101288.225169,101423.330533,101252.028661,101283.744069,100990.767851,101636.278058,101526.665823,101270.923669,101395.663386,101106.021647,101230.834429,101329.906200,101150.308808,101166.533230,101230.751642,101132.540772,101528.139285,101377.031351,100923.811857,101604.373985,101049.237882,101242.516067,100824.682970,101787.027009,101384.344796,101490.700045,101503.712399,101280.513099,101118.937621,101189.627931,100884.290201,101244.246182,101438.239518,100961.309538,100942.574317,101333.017692,100642.469954,100968.920927,100928.163964
1197,101424.824560,101607.553021,101519.466214,101315.346782,101555.770793,100923.208817,100812.566733,101136.832049,101294.587161,101344.577203,101542.826685,101302.702224,101483.424415,101283.581300,101019.263348,101572.934876,101668.461484,100851.191657,101242.127888,101288.655929,100990.325436,101300.692114,101346.225713,101167.941570,100991.729642,101325.585867,101089.850184,101169.336647,101114.847955,101263.172978,101062.390452,101133.340078,101340.558777,101452.488165,100637.268669,101310.392148,101372.477750,101452.087519,100811.760837,101320.617265,...,101130.734175,101497.796160,100961.736382,100897.377347,101442.170385,100942.640961,101305.002152,100993.381857,101032.315302,100669.047314,100816.323459,101260.899753,101242.837252,101279.942147,100892.515454,101616.906152,101154.476174,101209.642408,101164.888988,101116.385004,101195.046640,101275.507650,101029.264424,101398.581430,101386.515086,101187.344271,101865.566661,101618.340367,100925.661171,101259.165000,101269.157531,101192.083771,101203.424796,101402.633110,100971.088296,101042.098676,101009.517295,101093.191076,100908.374976,101218.192412
1198,101411.494394,101231.965522,101680.918101,101503.374825,100981.806134,101718.318530,101395.138359,101024.961402,101293.167701,101140.095614,101030.531093,101086.018674,101630.190758,101314.454564,101226.150451,100998.017761,101390.340477,101341.656123,101492.543774,101419.817895,101113.321119,101136.843773,101106.478279,101193.3173

###### *Growth Portfolio Simulation*

In [56]:
# Calculates portfolio expected returns from the portfolio ratios and the predicted returns 

stb_d1 = stb_abs_pred * stb_abs4
stb_d2 = stb_etf_pred * stb_etf1

# 100000 is the invested amount
grow = stb_d1.add(stb_d2)
growth = grow * 100000

#growth.insert(loc = index, column = 'Date', value = future_dates.strftime('%Y-%m'))

grow.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
0,1.051768,0.982230,1.021250,0.999574,1.009717,1.002545,0.996660,0.986669,0.990293,1.030095,0.991585,0.995362,1.006124,1.021836,0.969868,1.025635,0.993174,0.996600,0.978312,1.020539,0.998642,1.029969,1.011818,1.004539,1.028467,1.038373,0.978661,1.007131,0.993920,1.012790,0.984173,1.002599,1.009128,1.005597,1.013050,1.007074,1.015834,0.997090,0.985702,0.975985,...,1.010022,1.008316,1.005018,0.995453,1.006404,1.022068,1.009238,0.992817,1.034525,0.990422,1.008598,1.011849,1.015481,0.991976,0.998492,0.992916,1.010679,0.993399,1.010180,0.982760,1.028912,1.002768,0.992630,1.047983,0.997054,1.012575,1.002119,1.018636,1.047120,1.042746,1.032461,1.025523,1.000100,1.005753,1.013467,1.020163,0.988943,1.009836,1.021159,1.023480
1,0.985250,1.020658,1.006309,1.039878,0.984473,1.029727,0.999572,1.005414,1.032898,1.058850,1.001224,0.990605,1.011591,1.027752,1.019017,1.016886,1.029554,0.981033,1.004110,1.026511,1.024572,0.988145,1.044568,1.023413,1.032771,0.977133,1.014689,1.031954,0.996345,1.024851,1.031581,1.007303,0.986081,1.020466,1.003979,1.009756,1.024852,0.996113,0.993530,1.006105,...,0.982362,1.019046,0.998151,1.013399,0.998666,1.017287,0.999034,1.002902,1.039416,1.020134,1.017182,1.034891,1.003457,0.988137,1.003825,0.993174,1.016073,1.012264,1.009039,1.009836,1.005439,0.989085,0.968110,1.021395,1.006580,1.001513,0.974709,0.996492,1.007442,1.011495,1.010166,1.021603,0.991778,1.000416,1.000267,1.037334,1.001729,1.000931,1.007718,1.015328
2,0.981844,1.014634,1.014020,1.000464,1.006991,1.014356,1.046742,0.990763,1.022404,1.008787,1.013317,1.017226,0.995173,1.002956,1.032782,0.974140,1.018191,1.026543,1.009956,1.021208,0.993323,1.050283,0.999201,0.995670,1.033952,1.019199,1.000865,0.984175,0.989288,0.974147,1.014851,1.053765,0.994107,1.002867,1.026934,1.032719,0.977548,0.977621,0.988353,0.990917,...,0.988089,1.003706,1.041963,1.045332,1.000693,1.007610,1.017350,1.038432,0.997202,1.021604,1.011840,1.022532,0.996223,1.038316,1.032263,1.013217,0.987899,1.009173,0.998132,1.041575,1.010064,1.018519,1.016110,1.024871,1.008366,0.999443,0.985402,0.991024,1.003278,1.022333,0.993689,0.975861,1.023364,1.005045,0.991638,1.001395,1.017104,1.038419,1.021410,1.016708
3,0.993993,1.004176,1.006043,0.970532,0.994561,1.036640,1.029214,1.015028,1.020167,1.017023,0.997564,0.999737,0.995354,1.030536,0.990557,1.016537,0.975607,0.985995,1.004497,0.989915,1.000326,1.008306,1.000513,1.010544,1.011472,0.981456,1.021841,1.020099,0.993369,1.015055,1.017262,1.011987,1.017743,0.997591,1.035220,0.973979,0.993847,0.989924,1.018202,1.002261,...,1.011755,1.023165,0.991935,0.999614,1.006685,0.999579,1.003096,1.038688,0.993844,1.051417,1.015427,1.029381,0.982179,1.035649,1.005957,1.015245,1.019940,0.997244,0.996000,1.019754,1.005855,0.991794,1.015766,0.984647,1.011879,1.002165,0.980540,1.030525,1.023960,0.970382,1.008516,1.036555,1.000309,1.053656,0.985837,0.997748,0.965652,1.016550,1.008321,1.005864
4,1.002280,0.982443,1.064300,0.999009,0.988574,1.007943,1.027206,1.023808,1.018020,1.018751,1.017273,0.978656,1.025709,1.025056,1.091953,1.017606,0.987220,1.008298,0.996454,1.006661,0.989458,1.037296,0.982543,0.989338,1.010745,0.985581,1.002459,1.026880,1.025861,1.013643,1.021922,1.048712,1.013562,1.033286,1.025176,1.024079,0.979362,1.037858,1.015973,1.019717,...,0.994871,1.016540,1.017141,1.029954,1.023679,1.010343,1.043822,0.990911,1.009420,1.025354,1.005498,1.024734,1.015420,1.019636,1.003393,1.011177,1.009134,1.003051,1.029633,1.006184,1.024161,1.006889,0.990337,1.012472,0.995981,0.994461,1.012401,1.017017,1.046838,0.990521,1.042294,1.010896,1.018169,0.996421,1.015306,1.001934,1.024442,1.018746,1.032041,1.014899


In [57]:
grow.tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
1195,1.015644,1.031685,0.989601,1.015350,1.010354,0.972176,1.019466,1.018711,1.020783,1.049095,0.980306,1.028840,1.007356,1.030510,1.014602,1.000408,1.000084,1.017468,1.008796,1.012177,0.968150,1.033069,1.003043,0.987701,0.987233,1.018543,1.010245,1.021966,0.987604,1.040939,1.012368,1.034695,1.039502,1.014953,1.001790,0.991406,1.051698,0.996832,1.002330,1.014696,...,0.996963,1.000147,0.975187,1.033975,1.028340,1.003726,1.042055,0.992261,0.962624,0.992913,1.015765,1.023192,1.010848,0.983864,1.035845,1.001327,1.004783,0.988386,1.041809,0.988119,1.017081,1.042742,1.011865,1.004873,1.010466,1.016626,1.035464,1.001508,1.030669,1.007303,1.023989,1.025465,1.015064,1.004330,1.030382,1.043680,0.998223,0.978292,0.983598,1.018138
1196,1.014876,1.031295,1.007822,1.004949,1.018185,1.000542,1.009730,1.068487,1.048842,1.021187,1.005280,0.993763,1.025817,1.010882,1.002976,1.019052,1.034044,1.012604,0.984537,1.007084,1.001689,0.995410,1.005429,1.044165,1.000749,0.991283,1.001549,1.017341,1.027907,1.011014,1.017722,1.014560,1.000821,1.023240,1.045335,1.002479,1.004228,1.003350,1.013395,0.975389,...,1.012238,0.981120,1.043579,0.974587,1.025768,1.014487,1.057814,1.037914,0.985008,1.059210,1.041524,1.001472,1.036645,0.992795,1.040249,1.022910,0.986974,1.033958,1.015353,0.994133,1.007674,1.010150,1.022574,0.995265,1.020719,1.031632,1.029493,1.055078,0.982099,1.008532,1.036788,0.985183,1.016713,1.022393,1.009626,1.011604,1.036925,0.994436,0.976323,1.018352
1197,1.004975,0.991858,1.005969,1.010585,1.008855,1.014217,1.023117,1.008463,1.026410,0.991671,1.007098,1.004941,0.997863,0.985461,0.983230,1.008640,1.022776,1.029895,1.021099,1.015259,1.034148,1.011533,1.017114,1.005436,1.010791,1.010276,1.008106,0.995781,1.028931,1.009477,1.052342,1.046738,1.013461,1.020257,1.006790,0.988301,1.021269,1.005248,1.007638,1.028551,...,1.009286,1.018270,1.003717,0.995130,1.018767,0.960383,0.990084,0.990482,1.041848,1.022713,0.988119,0.978642,1.010069,1.023125,1.001509,1.013032,0.990187,0.991232,1.001681,1.010898,1.004556,0.999646,1.027358,1.017410,1.016416,1.002315,0.994437,0.987187,1.016672,1.010694,0.996558,0.981910,0.985328,1.021600,1.017365,1.022395,0.979397,1.010317,0.992774,1.008178
1198,0.990178,1.028859,1.014422,0.994336,1.013962,1.030215,1.005999,1.049388,1.012041,1.020432,0.968110,0.999819,1.020736,0.998491,0.993621,1.026884,0.994338,1.025573,1.048566,1.013346,1.030880,1.022643,0.990547,1.065294,1.023394,0.991965,1.011217,1.019427,1.026540,1.041470,1.016267,1.013916,0.981313,0.996616,1.007290,0.964126,1.020227,0.989149,0.988742,0.987140,...,1.014261,1.029189,1.014827,1.004919,1.006286,1.000266,1.024627,1.036838,0.996069,1.023330,0.987694,0.989509,0.998704,1.029032,0.988931,1.007517,0.999611,0.980367,1.012542,1.015819,0.992708,0.991427,1.009215,1.015821,1.003427,1.000583,1.020734,1.013420,0.990262,1.000604,0.993094,1.010237,1.010365,1.021838,1.021879,0.997289,1.029685,1.033445,1.017910,1.022361
1199,1.025583,1.027020,1.005557,0.986518,1.000042,0.967459,0.991135,1.006242,1.009770,1.017626,1.007445,1.002808,1.017129,0.991846,1.017310,1.048279,1.026060,1.020997,1.021894,1.044101,1.025544,1.028870,1.010895,1.047408,1.012399,1.010134,0.994564,1.007739,1.012259,0.991978,1.019489,0.986378,1.009792,1.014193,1.047701,1.018733,0.977889,1.023542,1.030472,0.991169,...,1.007885,1.034357,1.009634,1.037082,0.981211,0.994618,0.985880,0.974445,1.030212,0.981944,1.030637,0.980794,1.023262,1.000132,0.997958,1.028566,1.016424,1.017862,0.999876,1.029764,1.044952,1.003810,0.999991,1.015413,0.981171,1.008020,0.973159,1.007588,1.043102,1.020400,1.016187,1.017184,1.008271,1.010714,0.994296,0.995648,0.996827,1.023946,1.000428,0.980022


In [58]:
growth.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
0,105176.798267,98223.002724,102124.976348,99957.358868,100971.687296,100254.450557,99666.025463,98666.906149,99029.331197,103009.510652,99158.462853,99536.160242,100612.356674,102183.567551,96986.822836,102563.493051,99317.424221,99660.013989,97831.167753,102053.850234,99864.179880,102996.909133,101181.760049,100453.889225,102846.728661,103837.332224,97866.127648,100713.075242,99392.040844,101278.974173,98417.336102,100259.936176,100912.831553,100559.666277,101304.979202,100707.446945,101583.440188,99708.985226,98570.225314,97598.488576,...,101002.191587,100831.577393,100501.769248,99545.256228,100640.443416,102206.796258,100923.823939,99281.719640,103452.504282,99042.244525,100859.799105,101184.934568,101548.141280,99197.586281,99849.226406,99291.606085,101067.925215,99339.939159,101018.019559,98275.952202,102891.182523,100276.802547,99262.970284,104798.297165,99705.444825,101257.538539,100211.894444,101863.624894,104711.950321,104274.558832,103246.061313,102552.272804,100010.041734,100575.336914,101346.654658,102016.282314,98894.273068,100983.631452,102115.868037,102347.951066
1,98525.020535,102065.837293,100630.935614,103987.772220,98447.324236,102972.692380,99957.155778,100541.421294,103289.766759,105884.974778,100122.401591,99060.465519,101159.072305,102775.206283,101901.747534,101688.561327,102955.395362,98103.286316,100410.975044,102651.106280,102457.226701,98814.493423,104456.815749,102341.289902,103277.137046,97713.264440,101468.872155,103195.380143,99634.493658,102485.129342,103158.095396,100730.250922,98608.129289,102046.601211,100397.895369,100975.600305,102485.188198,99611.281256,99353.014046,100610.503175,...,98236.214446,101904.644730,99815.115896,101339.929934,99866.609799,101728.727393,99903.445724,100290.185695,103941.585211,102013.432148,101718.212293,103489.074995,100345.689114,98813.694104,100382.454735,99317.435086,101607.329374,101226.384033,100903.923078,100983.614542,100543.947901,98908.547002,96810.972812,102139.494719,100657.958221,100151.282770,97470.922139,99649.155886,100744.226067,101149.477967,101016.622333,102160.252109,99177.847848,100041.614095,100026.732555,103733.432669,100172.901850,100093.117952,100771.751054,101532.842352
2,98184.365464,101463.434552,101401.973695,100046.427367,100699.098598,101435.605646,104674.205589,99076.309615,102240.365700,100878.697776,101331.688989,101722.630318,99517.310341,100295.632450,103278.187074,97414.020712,101819.067788,102654.330897,100995.627157,102120.830931,99332.319389,105028.339350,99920.121134,99567.038106,103395.175442,101919.925646,100086.480755,98417.510595,98928.848942,97414.680976,101485.140905,105376.532511,99410.657315,100286.669603,102693.358989,103271.917415,97754.827683,97762.057203,98835.311185,99091.736223,...,98808.860236,100370.582904,104196.298330,104533.234805,100069.331246,100760.966884,101734.999024,103843.221236,99720.178646,102160.420571,101183.984277,102253.179976,99622.346641,103831.591427,103226.340401,101321.685280,98789.932623,100917.326061,99813.198975,104157.461916,101006.357109,101851.924357,101610.978737,102487.085352,100836.567908,99944.275000,98540.227843,99102.361295,100327.848433,102233.251015,99368.946072,97586.121678,102336.394190,100504.479316,99163.844739,100139.450083,101710.384103,103841.942600,102140.983702,101670.793321
3,99399.338516,100417.569884,100604.328909,97053.237428,99456.103824,103664.010555,102921.427574,101502.816937,102016.748517,101702.313822,99756.416859,99973.694781,99535.397499,103053.582357,99055.718121,101653.667437,97560.734507,98599.498806,100449.745572,98991.516277,100032.583562,100830.576685,100051.335869,101054.429514,101147.237812,98145.604140,102184.140660,102009.868773,99336.877061,101505.547734,101726.246380,

In [59]:
growth.tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
1195,101564.443727,103168.539972,98960.091348,101535.012717,101035.359009,97217.637627,101946.602668,101871.093331,102078.328841,104909.457937,98030.581345,102884.048853,100735.640669,103051.025358,101460.174657,100040.767134,100008.396694,101746.798989,100879.611357,101217.703503,96814.990715,103306.945881,100304.281072,98770.129894,98723.252626,101854.297016,101024.507854,102196.648041,98760.382676,104093.885690,101236.766337,103469.501869,103950.172372,101495.278993,100179.042684,99140.615014,105169.842525,99683.184968,100233.021966,101469.565422,...,99696.272476,100014.738240,97518.696769,103397.503018,102833.965832,100372.589290,104205.546709,99226.116481,96262.408950,99291.266172,101576.453589,102319.165547,101084.845738,98386.398248,103584.462478,100132.675101,100478.285030,98838.634196,104180.867965,98811.942434,101708.139702,104274.205087,101186.485765,100487.315600,101046.556692,101662.580469,103546.379383,100150.799026,103066.933679,100730.328424,102398.859879,102546.475704,101506.353556,100432.999963,103038.197205,104368.035933,99822.258337,97829.168100,98359.799136,101813.772965
1196,101487.620879,103129.469583,100782.169439,100494.890992,101818.487339,100054.226361,100972.986236,106848.698916,104884.170809,102118.680716,100528.031196,99376.250911,102581.684830,101088.225090,100297.593658,101905.198589,103404.370263,101260.443494,98453.684116,100708.410447,100168.949785,99540.994189,100542.853102,104416.480075,100074.930823,99128.319882,100154.889713,101734.122074,102790.719243,101101.421834,101772.153121,101456.033932,100082.124929,102324.008987,104533.475226,100247.855852,100422.788209,100335.040751,101339.465120,97538.884750,...,101223.825837,98111.989703,104357.850317,97458.681337,102576.829472,101448.716975,105781.408364,103791.359998,98500.786896,105921.018925,104152.435949,100147.164458,103664.488542,99279.520046,104024.934636,102290.983215,98697.385043,103395.765830,101535.305687,99413.289391,100767.394679,101015.006029,102257.364494,99526.505305,102071.898606,103163.179027,102949.277386,105507.801968,98209.944987,100853.178279,103678.809601,98518.291456,101671.289748,102239.316456,100962.581466,101160.392182,103692.515082,99443.625909,97632.280293,101835.193348
1197,100497.494505,99185.845722,100596.947543,101058.540024,100885.484222,101421.723377,102311.664842,100846.268723,102640.998447,99167.094540,100709.770201,100494.088945,99786.340592,98546.127387,98322.993408,100863.981121,102277.574021,102989.516227,102109.942107,101525.932693,103414.790358,101153.336852,101711.435895,100543.580360,101079.121778,101027.566673,100810.635475,99578.077322,102893.092801,100947.668277,105234.249062,104673.799040,101346.057808,102025.671530,100679.039356,98830.125570,102126.919915,100524.796054,100763.752925,102855.122388,...,100928.565007,101826.997778,100371.698490,99513.008363,101876.691931,96038.302030,99008.437712,99048.249207,104184.830429,102271.320246,98811.869283,97864.247908,101006.886473,102312.467104,100150.879066,101303.221181,99018.695880,99123.239272,100168.102716,101089.764576,100455.584064,99964.610156,102735.781412,101740.999813,101641.578221,100231.517018,99443.720631,98718.703623,101667.151543,101069.443708,99655.835589,98191.010046,98532.829085,102160.044957,101736.512084,102239.494968,97939.678261,101031.749867,99277.354801,100817.788703
1198,99017.771473,102885.856241,101442.162665,99433.599212,101396.212879,103021.478155,100599.929252,104938.799150,101204.074302,102043.230032,96811.028311,99981.929334,102073.644152,99849.050909,99362.128753,102688.433594,99433.790493,102557.273395,104856.571968,101334.648848,103087.955330,102264.281304,99054.688600,106529.390596,102339.429484,99196.536838,101121.697356,101942.694752,102653.98

###### Aggressive Portfolio Simulation

In [60]:
# Calculates portfolio expected returns from the portfolio ratios and the predicted returns 

stb_e1 = stb_abs_pred * stb_abs5
stb_e2 = stb_etf_pred * stb_etf2

# 100000 is the invested amount
aggres = stb_e1.add(stb_e2)
aggressive = aggres * 100000

#aggressive.insert(loc = index, column = 'Date', value = future_dates.strftime('%Y-%m'))

aggres.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
0,1.085874,0.961509,1.027823,0.985547,1.005715,0.994850,0.986829,0.964331,0.973498,1.044853,0.976529,0.980986,0.998184,1.027791,0.935152,1.033566,0.977334,0.986886,0.951472,1.024609,0.989682,1.041773,1.012381,0.998964,1.044482,1.061950,0.949260,0.999151,0.977741,1.012685,0.960970,0.994847,1.006867,0.998470,1.014037,1.002342,1.017163,0.983347,0.964821,0.941057,...,1.009505,1.006773,1.002236,0.979338,0.998883,1.032567,1.009435,0.973041,1.050235,0.974055,1.001427,1.010233,1.016117,0.975539,0.987911,0.980523,1.012254,0.973585,1.010444,0.962388,1.044028,0.992790,0.975638,1.078462,0.985851,1.012144,0.994618,1.024297,1.074914,1.063345,1.047920,1.038385,0.987898,1.000522,1.014238,1.025339,0.972065,1.007860,1.030758,1.030243
1,0.961140,1.026936,1.002813,1.062139,0.967496,1.044802,0.991948,1.002259,1.050799,1.098147,0.989612,0.977375,1.008328,1.041549,1.023232,1.021408,1.041834,0.957202,0.996700,1.036756,1.035450,0.969241,1.072094,1.029510,1.047211,0.950240,1.018469,1.047362,0.983198,1.034030,1.040652,1.001588,0.964372,1.024720,0.996688,1.006682,1.037733,0.981944,0.978700,0.999903,...,0.959179,1.024554,0.988266,1.015958,0.987059,1.022487,0.989040,0.992749,1.063074,1.027175,1.024927,1.056545,0.998587,0.966666,0.998368,0.976753,1.020359,1.011483,1.006701,1.009621,0.996795,0.968734,0.931389,1.026990,1.005136,0.995182,0.941849,0.985206,1.007063,1.006732,1.010280,1.031342,0.970747,0.991103,0.988355,1.053081,0.992984,0.993981,1.005479,1.015398
2,0.955753,1.016446,1.014512,0.989910,1.003860,1.013029,1.078193,0.971188,1.029832,1.008071,1.011797,1.021618,0.978311,0.997269,1.048800,0.940883,1.025547,1.035326,1.007816,1.027008,0.978968,1.079913,0.992046,0.981258,1.047760,1.025350,0.991478,0.960832,0.971370,0.945460,1.019044,1.080714,0.979072,0.998370,1.041049,1.050401,0.952750,0.949685,0.968684,0.971288,...,0.966056,0.996516,1.061432,1.069108,0.992760,1.003734,1.017683,1.056918,0.985619,1.028236,1.006246,1.032062,0.982549,1.059072,1.046769,1.015175,0.969745,1.009224,0.985309,1.066138,1.009032,1.022480,1.013753,1.034404,1.005388,0.986012,0.962226,0.975508,0.997650,1.029007,0.975273,0.944018,1.034304,0.997032,0.973755,0.986315,1.017877,1.058935,1.030253,1.017622
3,0.982284,0.998928,0.998795,0.933284,0.979936,1.057645,1.042582,1.016031,1.026969,1.020085,0.982627,0.992228,0.981426,1.050123,0.972447,1.017760,0.943047,0.962525,0.998784,0.969221,0.989740,1.002695,0.988912,1.004596,1.012103,0.956310,1.029117,1.030586,0.980324,1.016237,1.020627,1.016167,1.020110,0.983580,1.051832,0.938406,0.977758,0.971693,1.022652,0.990907,...,1.014626,1.035274,0.975890,0.994166,1.002197,0.990470,0.994033,1.058578,0.978104,1.081291,1.019293,1.040927,0.958033,1.056773,1.002507,1.018508,1.023995,0.986037,0.982806,1.025906,0.997899,0.972851,1.017414,0.964600,1.012808,0.993266,0.955595,1.044685,1.036886,0.933456,1.000079,1.052268,0.995359,1.087275,0.968933,0.985905,0.925637,1.024454,1.004097,1.000720
4,0.993836,0.958563,1.103755,0.991481,0.964712,1.007562,1.036469,1.030100,1.020016,1.024221,1.023983,0.948315,1.038353,1.038837,1.152686,1.022197,0.965741,1.000051,0.979727,1.001270,0.971923,1.054903,0.956658,0.969320,1.016022,0.963082,0.990841,1.038292,1.037216,1.015845,1.029487,1.079231,1.017512,1.050821,1.036111,1.032890,0.949505,1.054941,1.018171,1.020448,...,0.977822,1.022188,1.023169,1.041177,1.029780,1.009283,1.064984,0.974412,1.008180,1.037399,0.999227,1.028255,1.016134,1.024799,0.998255,1.011150,1.008745,0.995613,1.041333,1.001027,1.032121,1.003382,0.969413,1.013111,0.979109,0.981295,1.014987,1.019724,1.073529,0.972223,1.069651,1.012040,1.023695,0.982313,1.009191,0.991964,1.035144,1.024971,1.049199,1.021482


In [61]:
aggres.tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
1195,1.015284,1.045808,0.968095,1.018251,1.008629,0.940077,1.026205,1.024358,1.026939,1.078653,0.956117,1.041443,1.001869,1.045474,1.015469,0.992539,0.988992,1.017631,1.007947,1.014414,0.931841,1.045031,0.994408,0.962820,0.968421,1.022193,1.009926,1.029970,0.965119,1.063395,1.009473,1.050668,1.061510,1.017235,0.987050,0.978792,1.083390,0.979869,0.995288,1.017301,...,0.983791,0.990971,0.943468,1.050015,1.038148,0.992897,1.070223,0.979682,0.925567,0.974349,1.021724,1.034248,1.009404,0.961872,1.053871,0.992137,1.004060,0.970228,1.067971,0.974347,1.018390,1.065985,1.009911,0.998161,1.008223,1.021934,1.051475,0.989559,1.050273,1.005134,1.035360,1.030959,1.015277,0.996773,1.045027,1.070951,0.987282,0.948741,0.958067,1.021069
1196,1.014649,1.043115,1.001515,1.000702,1.027145,0.990695,1.004620,1.113617,1.073328,1.031425,0.998671,0.979217,1.035428,1.012047,0.994788,1.020596,1.055987,1.010659,0.964558,1.005626,0.991779,0.981930,0.995270,1.068787,0.991854,0.973555,0.993693,1.020840,1.038007,1.012459,1.020658,1.009579,0.990783,1.030191,1.071777,0.994228,0.997406,0.994166,1.011827,0.942556,...,1.016375,0.955196,1.067606,0.943066,1.036186,1.018397,1.090773,1.055977,0.961965,1.095712,1.066816,0.992102,1.055431,0.977798,1.063600,1.031078,0.966931,1.048752,1.016708,0.982315,1.000181,1.010144,1.030439,0.984837,1.022466,1.046123,1.041245,1.087443,0.956779,1.006204,1.057372,0.965414,1.019997,1.028953,1.010337,1.013641,1.055549,0.985414,0.949072,1.026368
1197,0.996764,0.971695,0.997850,1.007725,1.002971,1.018481,1.036082,1.005594,1.036558,0.973571,0.999768,0.998239,0.983551,0.963341,0.961503,1.002540,1.027317,1.047684,1.027977,1.017291,1.053648,1.010114,1.020049,1.000301,1.011452,1.007325,1.005786,0.982773,1.043104,1.006532,1.086236,1.075616,1.013025,1.024792,1.008187,0.967738,1.027299,0.997735,1.007292,1.041019,...,1.007205,1.020492,0.999463,0.984032,1.021857,0.920554,0.971020,0.974869,1.067805,1.036018,0.971722,0.950777,1.007774,1.031769,0.995405,1.010025,0.972941,0.973546,0.993014,1.011011,0.998556,0.989020,1.040897,1.020014,1.018775,0.994563,0.974020,0.963329,1.022714,1.008571,0.983394,0.957528,0.963404,1.027714,1.023737,1.032127,0.954423,1.010181,0.979598,1.004839
1198,0.970491,1.041955,1.011952,0.977244,1.017430,1.040703,0.999185,1.081437,1.010990,1.027782,0.933627,0.990782,1.023716,0.986152,0.978107,1.041403,0.977640,1.034881,1.075481,1.012660,1.047489,1.031747,0.974320,1.108683,1.028029,0.974003,1.010579,1.025967,1.039891,1.064651,1.019611,1.012560,0.957477,0.981752,1.003827,0.925910,1.025668,0.972102,0.967660,0.964840,...,1.016096,1.039653,1.016354,0.996430,1.003393,0.991004,1.028157,1.056457,0.982551,1.032817,0.967607,0.969574,0.985428,1.045634,0.965945,1.004009,0.986277,0.955029,1.014968,1.017291,0.976869,0.974215,1.011076,1.020010,0.993678,0.994791,1.024236,1.014062,0.972366,0.989777,0.976239,1.011981,1.004110,1.030274,1.026443,0.981924,1.044111,1.050730,1.022703,1.030520
1199,1.035758,1.040693,0.999008,0.966389,0.989946,0.932379,0.967933,1.001655,1.007476,1.021584,1.005395,0.994569,1.021508,0.977307,1.020954,1.080288,1.034819,1.030048,1.031798,1.069464,1.037629,1.039984,1.008877,1.071850,1.009577,1.012730,0.980145,1.005902,1.008113,0.974531,1.026789,0.961850,1.007424,1.015976,1.075375,1.020524,0.945340,1.034074,1.044074,0.978206,...,1.003457,1.051062,1.006085,1.054355,0.957115,0.981803,0.961393,0.944277,1.040015,0.958489,1.044596,0.955413,1.032969,0.988196,0.985827,1.043727,1.020024,1.025168,0.993182,1.047407,1.070824,0.996413,0.994089,1.018106,0.959765,1.003331,0.942737,1.002843,1.068937,1.024973,1.017415,1.018464,1.007789,1.007447,0.976521,0.984672,0.988167,1.034250,0.992285,0.955796


In [62]:
aggressive.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
0,108587.391608,96150.883221,102782.326883,98554.707593,100571.534991,99485.016144,98682.851259,96433.069020,97349.843062,104485.276308,97652.917645,98098.574950,99818.411583,102779.087359,93515.184919,103356.576736,97733.395249,98688.558731,95147.206520,102460.948643,98968.248596,104177.334208,101238.122110,99896.389952,104448.237374,106195.024546,94926.012219,99915.117528,97774.061381,101268.459010,96097.007558,99484.689130,100686.723343,99846.993696,101403.681517,100234.218807,101716.266700,98334.741989,96482.066679,94105.704662,...,100950.505023,100677.331084,100223.569997,97933.765057,99888.305703,103256.687976,100943.529192,97304.139241,105023.470524,97405.507196,100142.662035,101023.292176,101611.715529,97553.858700,98791.059072,98052.288765,101225.409283,97358.536611,101044.422265,96238.799872,104402.839088,99279.040407,97563.806569,107846.175557,98585.109121,101214.448631,99461.772151,102429.689940,107491.395480,106334.465267,104792.032184,103838.491460,98789.774592,100052.192659,101423.792098,102533.938201,97206.539745,100786.030714,103075.760424,103024.320594
1,96114.017674,102693.578427,100281.250463,106213.946846,96749.626725,104480.188769,99194.772726,100225.918437,105079.932669,109814.749200,98961.186316,97737.488960,100832.804398,104154.889552,102323.213184,102140.767706,104183.356092,95720.248016,99669.984328,103675.557364,103544.960500,96924.097539,107209.372628,102951.016037,104721.132097,95023.976484,101846.875358,104736.210957,98319.822160,103403.045001,104065.220588,100158.801355,96437.211366,102471.988898,99668.783983,100668.225388,103773.271597,98194.400763,97870.020861,99990.343449,...,95917.860901,102455.380145,98826.630906,101595.822119,98705.894279,102248.653228,98904.007883,99274.861563,106307.379316,102717.518162,102492.719090,105654.490807,99858.696366,96666.634393,99836.818706,97675.291475,102035.928278,101148.268344,100670.105494,100962.138743,99679.501298,96873.398234,93138.931809,102698.997161,100513.639890,99518.206163,94184.866345,98520.557744,100706.318688,100673.162477,101028.019510,103134.164758,97074.732083,99110.266745,98835.470258,105308.078079,99298.386800,99398.084580,100547.869519,101539.813336
2,95575.257083,101644.586780,101451.228143,98990.986684,100385.966115,101302.866121,107819.336873,97118.786041,102983.153361,100807.078134,101179.731429,102161.827146,97831.129292,99726.943712,104879.961632,94088.310173,102554.697903,103532.612122,100781.602857,102700.793089,97896.844791,107991.297417,99204.643227,98125.775353,104775.972072,102535.029496,99147.809650,96083.188831,97136.990340,94545.957463,101904.400317,108071.378474,97907.179625,99837.045023,104104.904477,105040.130791,95274.974112,94968.484741,96868.370496,97128.786629,...,96605.552557,99651.625403,106143.242941,106910.813264,99276.032089,100373.388179,101768.346952,105691.807333,98561.943839,102823.569896,100624.624691,103206.208965,98254.877684,105907.150648,104676.930274,101517.491995,96974.471811,100922.407575,98530.896093,106613.807908,100903.203634,102248.023678,101375.281428,103440.431666,100538.779713,98601.187852,96222.649179,97550.763499,99764.960657,102900.731423,97527.333515,94401.803201,103430.386721,99703.240629,97375.466962,98631.467717,101787.681170,105893.467017,103025.347770,101762.202178
3,98228.402647,99892.769752,99879.513512,93328.367170,97993.649008,105764.534161,104258.247735,101603.094527,102696.875679,102008.455871,98262.718557,99222.816991,98142.577629,105012.282537,97244.662074,101775.961765,94304.693700,96252.486385,99878.440936,96922.136716,98974.018195,100269.527560,98891.186788,100459.580996,101210.282583,95631.049665,102911.741423,103058.583162,98032.371732,101623.728300,102062.734218,101616.725212,102010.987960,98358.042

In [63]:
aggressive.tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
1195,101528.444304,104580.761490,96809.503378,101825.103135,100862.909222,94007.657645,102620.508665,102435.830634,102693.864327,107865.270783,95611.678017,104144.287467,100186.872642,104547.375868,101546.929391,99253.880906,98899.230847,101763.082556,100794.663820,101441.360613,93184.127200,104503.069924,99440.775365,96281.969666,96842.064321,102219.267046,100992.620316,102997.019348,96511.916607,106339.534822,100947.347362,105066.770951,106150.991561,101723.520489,98705.015867,97879.210963,108339.038958,97986.893268,99528.752436,101730.065142,...,98379.064898,99097.124831,94346.795474,105001.546416,103814.812635,99289.736885,107022.338300,97968.234442,92556.741158,97434.919281,102172.425814,103424.762313,100940.428193,96187.238506,105387.056557,99213.667345,100406.023111,97022.816704,106797.055165,97434.650440,101838.994429,106598.512427,100991.113010,99816.069239,100822.253671,102193.363218,105147.478907,98955.869576,105027.342115,100513.357013,103535.982298,103095.911731,101527.699822,99677.258380,104502.681907,107095.076957,98728.193841,94874.140053,95806.651536,102106.873662
1196,101464.902246,104311.548685,100151.459230,100070.153668,102714.540010,99069.524419,100462.027138,111361.721391,107332.845943,103142.480035,99867.057843,97921.704902,103542.837115,101204.722678,99478.777370,102059.645993,105598.733023,101065.873582,96455.763706,100562.550975,99177.904193,98193.049744,99527.008495,106878.699272,99185.381063,97355.504180,99369.258584,102084.015182,103800.675059,101245.936425,102065.842525,100957.905614,99078.316581,103019.127876,107177.663025,99422.796758,99740.583457,99416.617771,101182.681768,94255.618336,...,101637.486302,95519.575693,106760.566087,94306.596941,103618.644882,101839.722218,109077.310078,105597.694911,96196.530882,109571.189367,106681.637128,99210.187822,105543.094310,97779.829324,106360.026660,103107.822110,96693.144901,104875.210883,101670.841175,98231.499883,100018.100586,101014.394337,103043.916114,98483.693849,102246.618892,104612.263501,104124.491815,108744.267992,95677.945972,100620.365707,105737.189749,96541.429751,101999.673800,102895.346242,101033.690477,101364.086762,105554.937198,98541.353741,94907.233355,102636.760556
1197,99676.427376,97169.535183,99785.032120,100772.468639,100297.111136,101848.094298,103608.225323,100559.370902,103655.823298,97357.136930,99976.761058,99823.860106,98355.059771,96334.056154,96150.299361,100253.955756,102731.700412,104768.374035,102797.662717,101729.118120,105364.849279,101011.436289,102004.875138,100030.147046,101145.157193,100732.478176,100578.639196,98277.306119,104310.413142,100653.199345,108623.620470,107561.601605,101302.530543,102479.176168,100818.731992,96773.838939,102729.932482,99773.514901,100729.177305,104101.869033,...,100720.509939,102049.200734,99946.288024,98403.153594,102185.704540,92055.363462,97102.031271,97486.864401,106780.490293,103601.750016,97172.238741,95077.670557,100777.370627,103176.856320,99540.488343,101002.532050,97294.129065,97354.642524,99301.377807,101101.076375,99855.561962,98902.017518,104089.722070,102001.436511,101877.487043,99456.288024,97402.015580,96332.934630,102271.364775,100857.090151,98339.391255,95752.811209,96340.447317,102771.399642,102373.714729,103212.684030,95442.298128,101018.148398,97959.844638,100483.853270
1198,97049.070283,104195.538812,101195.241733,97724.419608,101742.955818,104070.271749,99918.464096,108143.722125,101099.002589,102778.248140,93362.683028,99078.153678,102371.580222,98615.235252,97810.715919,104140.325938,97764.012919,103488.148321,107548.081587,101265.957526,104748.894498,103174.665561,97432.035279,110868.322775,102802.927324,97400.273567,101057.932972,102596.655911,103989.111510,106465.056143,101961.064311,1

###### *Dollar Portfolio Simulation*

In [64]:
# Calculates portfolio expected returns from the portfolio ratios and the predicted returns 

stb_f = stb_euro_pred * stb_euro
fbn_a = fbn_euro_pred * fbn_euro

# 100000 is the invested amount
doll = stb_f.add(fbn_a)
dollar = doll * 100000

#dollar.insert(loc = index, column = 'Date', value = future_dates.strftime('%Y-%m'))

doll.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
0,1.003904,1.000015,0.996278,1.005506,1.010898,1.007312,0.994053,1.010207,1.011194,0.999733,1.000891,1.004336,1.005444,0.994967,1.013431,0.989884,1.001761,0.999686,1.009673,1.000290,1.003155,1.004778,1.005512,1.002176,0.998083,0.997601,0.993692,1.009641,1.003812,1.001410,1.010944,1.009403,1.003339,1.003248,1.005415,0.999446,0.993548,0.994712,1.000396,0.998583,...,0.999501,0.998618,1.001653,1.010539,1.005425,1.001001,1.002353,1.010825,1.007435,1.014005,1.005555,0.992470,1.011109,1.009439,1.007027,0.994338,0.992357,1.000115,1.002682,0.998246,1.011860,1.003855,0.992606,1.007820,0.999748,1.003775,1.003363,1.003578,1.005243,1.003161,0.994450,1.014072,1.011534,1.005540,0.997996,1.004973,1.000962,0.998345,0.990598,0.992203
1,1.000350,1.001962,0.996375,0.998350,1.002287,1.015753,1.009828,0.997427,0.999283,0.995175,1.000070,1.008478,1.006779,1.000907,1.002144,1.004657,1.011420,0.998087,0.994804,1.005214,1.003011,1.004111,1.003335,1.015234,0.990998,1.014587,0.997976,1.007142,1.003963,1.007349,0.989653,1.013672,1.005026,0.992685,1.009667,0.997083,0.993399,1.007201,1.007704,0.996337,...,0.989469,1.008744,1.006785,0.991750,1.011325,1.003407,1.005510,1.002842,1.011470,1.002047,1.010016,0.998731,0.995857,1.002317,1.001576,1.007415,0.999269,0.992004,0.994160,1.011921,1.004271,1.002047,1.008128,1.005277,0.997348,0.988938,0.997792,0.990262,1.002841,0.998973,0.998964,1.013583,1.009002,0.998466,1.009293,1.003197,1.004529,1.000787,1.006036,1.006146
2,1.012114,0.999479,0.991758,0.995416,0.998640,1.006659,0.998970,1.001153,1.000645,0.997488,1.000164,1.014118,1.005138,1.005584,1.001331,1.012996,1.007096,0.991639,1.003585,0.997665,1.005695,1.002428,0.996036,1.007692,0.991810,1.009724,0.987973,1.006517,1.001298,0.992938,1.010885,1.003105,1.015666,1.009831,1.006797,1.003527,0.999645,1.001491,1.010442,0.996720,...,1.009659,1.012724,0.995139,1.003194,0.995725,0.998275,0.995248,1.007455,1.000650,1.000371,1.012713,0.999264,0.988152,1.006820,1.024501,1.000702,1.004973,1.002955,0.999377,0.994758,1.001457,1.004275,0.994308,1.007262,1.008716,0.990928,1.007802,1.001162,1.002542,1.023514,0.997401,1.011015,0.989978,1.012477,0.999572,1.000334,0.990204,1.006719,0.991683,0.995614
3,0.996906,0.998617,1.000046,1.005476,1.000307,1.003117,1.004373,1.004069,1.005221,1.012606,0.998412,0.995627,0.999405,1.009014,1.000065,1.001913,0.995773,1.011868,0.993518,1.001035,1.002183,1.007963,1.002005,1.005273,1.007937,1.002116,1.013988,1.002107,1.005468,1.006806,1.002779,1.001132,0.989846,1.002622,0.992487,1.000031,1.000361,0.999708,1.008756,1.001310,...,1.011112,1.000419,0.993509,1.005163,1.007307,1.005340,0.999005,1.009362,1.005054,1.004280,0.999110,0.995897,0.989816,1.007543,0.997429,1.003637,0.996919,0.994481,1.002714,1.010778,1.005836,1.012910,1.004503,1.016979,1.002063,1.013152,0.998253,0.996518,0.990017,1.013949,1.004551,1.005471,1.009130,1.015472,0.992731,1.007118,1.008899,1.000426,1.012301,1.000195
4,1.003646,1.006959,1.007199,0.998857,0.996080,0.989629,1.013502,1.014393,0.995010,0.992999,1.005314,0.999509,1.013272,1.005558,1.006219,0.983697,1.009726,1.008977,0.999349,1.002507,0.998842,0.998471,1.006166,1.003964,1.001756,1.006843,0.996924,1.009216,1.002923,0.999306,0.989898,0.997478,0.997070,0.995988,1.001062,1.002302,1.011103,0.992732,1.005693,0.996835,...,0.998927,1.000783,1.002788,1.011280,1.012640,1.007414,0.998999,1.011323,1.009535,1.002859,1.003823,1.012215,0.996242,1.000662,0.999398,0.999142,1.000200,1.013790,0.997879,1.005905,0.994565,1.001610,0.996270,1.012021,0.999913,0.996837,1.006727,0.998166,1.004957,1.011113,1.006664,1.006962,1.006895,1.005125,0.997735,1.000283,1.006269,1.007839,0.995971,1.006435


In [65]:
doll.tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
1195,0.991582,1.004435,1.007543,1.006649,0.999514,0.998615,1.001204,1.016471,1.011664,0.994509,1.013713,1.005131,1.005829,0.996042,1.015829,1.008788,1.005421,0.993874,1.002020,1.006174,1.000649,1.001283,1.000480,0.994737,1.001458,0.999872,0.997736,1.004735,1.006485,0.994389,1.004827,1.006317,1.011469,1.008756,1.013137,1.009002,1.000990,1.003467,1.008393,1.015049,...,0.998038,1.000528,1.006496,0.996683,1.002042,1.009876,1.001379,1.001044,1.004535,1.008935,0.996158,0.999247,1.000447,1.003271,0.996232,0.999008,1.004711,0.997638,1.002240,1.004681,1.006009,1.004614,0.997192,1.014527,1.007877,0.995003,1.005597,1.006468,1.006319,0.993341,1.007259,1.002774,1.012310,1.001713,0.992275,1.005612,1.007111,1.008530,1.003391,1.004070
1196,1.013195,1.004885,1.000954,0.988465,0.999297,0.998042,1.011974,0.989785,1.007686,1.006745,0.998879,1.003086,0.995413,0.998951,0.992607,1.010576,0.998503,0.997026,0.995763,1.002173,0.994992,1.006446,0.990937,1.005547,1.007484,0.999766,1.014601,0.997176,1.005993,1.008744,1.002289,1.001911,1.001322,0.998563,1.003198,0.991163,0.993655,0.997252,0.998768,1.014704,...,1.009018,1.014686,0.997690,1.007683,0.999134,1.007834,1.001085,1.006245,1.002282,1.001470,1.003121,1.000008,1.009398,1.008530,0.996807,1.003338,0.992719,0.992687,0.995748,1.005657,1.003089,0.994773,1.000677,1.002204,1.001888,1.001130,0.999858,1.007110,0.992292,1.006592,1.012881,0.995201,1.007376,0.992801,1.003535,1.002509,0.991676,1.007698,1.001058,0.987279
1197,0.999532,1.012426,1.005871,1.004916,0.995223,1.009021,1.007641,0.996747,0.996925,1.006928,1.007005,1.005209,1.003694,1.004938,0.999550,1.002859,0.996463,0.999276,1.000557,1.005668,0.996145,1.006669,1.010403,0.999603,1.006923,1.001037,1.004436,0.997462,1.008631,0.996935,1.010845,0.998301,1.004655,1.010034,1.001349,0.995648,1.009332,0.993887,1.005516,1.006482,...,1.008318,1.000814,1.000259,0.999582,1.002039,1.011625,0.990142,1.000930,1.002699,1.001022,0.999741,1.008075,0.997553,1.001506,0.997450,1.008761,0.997823,1.008154,1.000903,0.998079,1.002329,1.000359,1.013286,1.005854,0.996239,1.004702,1.007189,0.996846,1.000275,1.011608,1.010448,0.992065,1.006636,0.997653,0.994773,0.994397,0.993899,1.012291,1.018253,0.996201
1198,1.005132,1.004291,1.002662,0.995418,0.999725,0.995649,1.005851,1.000799,0.999743,0.990251,1.005246,1.003050,1.004934,1.000951,1.009204,1.013207,0.998704,1.009275,1.006426,0.986737,1.006302,0.995344,0.996779,1.005959,1.000326,1.005883,1.003614,1.002870,1.018480,1.009025,1.004828,1.001967,1.008699,1.006778,1.005417,1.003012,1.011620,0.999818,1.003133,1.005586,...,1.000577,1.003239,0.995176,0.999564,1.001866,1.005282,1.007307,1.000873,1.004183,1.013710,0.996804,0.997850,0.999235,1.003405,1.012799,1.003038,1.009658,0.996412,1.006180,1.005751,1.007959,1.002492,1.006603,1.016230,1.002482,0.995337,1.011245,1.001652,0.984394,1.009865,0.993403,1.009982,1.011821,1.011848,1.000252,0.999671,1.009073,0.992802,1.008116,0.999877
1199,0.999612,0.989824,1.010896,0.996858,0.996486,1.007762,0.997408,1.001727,1.013918,1.002016,1.009013,0.995323,1.002778,1.005838,1.002417,0.995984,1.006679,1.003359,0.998836,1.012624,1.006024,1.004456,1.002305,1.006585,0.992548,1.008372,1.001862,1.001988,0.999569,1.007258,1.002796,1.010375,1.006704,1.004792,0.996196,1.017237,0.998725,1.006383,1.000756,1.008774,...,0.997798,0.990504,1.010406,1.000207,0.991659,0.995758,1.018292,1.013213,1.002482,1.000520,1.001681,1.004654,1.006231,1.011534,0.999362,1.004626,1.009462,0.997768,1.017447,0.998958,1.006500,1.000718,1.004908,0.999047,1.006753,1.011146,0.998177,1.011102,1.008934,1.010867,1.009663,0.995974,1.006651,1.002863,0.997832,1.005567,1.018145,1.006395,1.006734,0.997585


In [66]:
dollar.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
0,100390.389891,100001.462968,99627.757780,100550.605977,101089.750213,100731.219036,99405.276937,101020.688666,101119.448508,99973.337030,100089.121516,100433.592377,100544.386456,99496.720095,101343.052634,98988.436668,100176.144513,99968.602063,100967.322943,100028.978180,100315.507638,100477.814336,100551.226679,100217.597271,99808.321163,99760.051618,99369.181857,100964.102652,100381.198878,100140.985141,101094.390101,100940.287292,100333.909854,100324.783183,100541.463593,99944.628237,99354.848636,99471.187032,100039.617361,99858.282921,...,99950.080752,99861.842569,100165.323395,101053.859724,100542.509466,100100.128271,100235.278243,101082.527363,100743.489647,101400.549463,100555.546469,99246.976228,101110.948016,100943.863705,100702.654180,99433.798515,99235.689050,100011.476241,100268.235540,99824.614067,101185.987283,100385.515870,99260.643995,100781.994047,99974.764089,100377.501319,100336.320335,100357.807400,100524.284515,100316.140258,99444.987024,101407.180061,101153.392327,100553.975833,99799.599494,100497.336521,100096.220330,99834.496028,99059.788870,99220.268296
1,100034.977913,100196.163049,99637.528690,99835.027978,100228.699055,101575.327076,100982.809931,99742.650059,99928.285705,99517.456196,100006.958998,100847.771483,100677.897661,100090.655317,100214.424016,100465.722574,101141.987400,99808.698047,99480.379232,100521.447086,100301.085871,100411.061935,100333.498779,101523.356745,99099.784852,101458.660679,99797.587790,100714.161141,100396.325242,100734.931928,98965.305232,101367.152507,100502.578075,99268.472060,100966.721091,99708.256351,99339.942244,100720.140217,100770.400835,99633.685318,...,98946.942268,100874.415480,100678.509909,99174.961191,101132.455614,100340.729629,100550.957958,100284.189790,101146.950760,100204.651265,101001.586092,99873.069387,99585.702529,100231.676424,100157.575821,100741.502397,99926.926546,99200.392490,99415.978479,101192.067820,100427.087892,100204.749994,100812.785443,100527.686195,99734.836999,98893.797681,99779.235325,99026.162560,100284.117457,99897.344087,99896.449382,101358.269869,100900.205542,99846.599396,100929.276245,100319.717029,100452.921009,100078.702292,100603.570555,100614.634369
2,101211.369899,99947.900651,99175.756250,99541.600633,99863.995233,100665.945080,99897.006596,100115.334939,100064.541855,99748.826923,100016.368118,101411.799934,100513.798366,100558.370320,100133.060560,101299.649549,100709.614911,99163.898161,100358.466926,99766.480319,100569.483538,100242.789506,99603.649732,100769.209001,99180.981864,100972.406800,98797.304834,100651.730878,100129.818994,99293.775618,101088.522345,100310.512810,101566.569351,100983.140333,100679.720310,100352.730980,99964.511193,100149.084037,101044.160766,99672.048248,...,100965.878368,101272.361167,99513.913558,100319.414146,99572.459643,99827.482274,99524.825874,100745.525527,100065.009766,100037.128450,101271.319570,99926.361543,98815.249938,100682.005630,102450.147102,100070.238259,100497.335134,100295.524398,99937.689582,99475.826556,100145.701371,100427.451936,99430.794869,100726.241368,100871.558505,99092.764294,100780.153424,100116.188378,100254.212026,102351.422143,99740.119505,101101.516069,98997.755772,101247.696190,99957.193488,100033.372779,99020.447645,100671.877191,99168.314214,99561.438175
3,99690.614181,99861.676112,100004.594783,100547.625101,100030.729078,100311.717358,100437.299379,100406.949640,100522.119511,101260.597195,99841.158237,99562.702036,99940.453388,100901.389221,100006.499537,100191.295836,99577.294147,101186.847177,99351.849852,100103.536943,100218.298005,100796.260898,100200.454004,100527.315964,100793.661055,100211.593916,101398.829391,100210.655944,100546.756252,100680.642978,100277.856601,10

In [67]:
dollar.tail()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999
1195,99158.196323,100443.496537,100754.270959,100664.893968,99951.356479,99861.515092,100120.407539,101647.072846,101166.371403,99450.938551,101371.304849,100513.076118,100582.940382,99604.205555,101582.881486,100878.813839,100542.137653,99387.380704,100201.992476,100617.401410,100064.949476,100128.301686,100047.957301,99473.662692,100145.815972,99987.224623,99773.623160,100473.478093,100648.513910,99438.863122,100482.652057,100631.723896,101146.895222,100875.616107,101313.692067,100900.235950,100099.037112,100346.718214,100839.250131,101504.935322,...,99803.840489,100052.754504,100649.596937,99668.306159,100204.244479,100987.629383,100137.890374,100104.430094,100453.484769,100893.487737,99615.828519,99924.708410,100044.730587,100327.051779,99623.182039,99900.827422,100471.118412,99763.833180,100224.032415,100468.121152,100600.855915,100461.422413,99719.235794,101452.682527,100787.699030,99500.305356,100559.681601,100646.802748,100631.897495,99334.093874,100725.874521,100277.428050,101230.962637,100171.319886,99227.451553,100561.179825,100711.081032,100852.958096,100339.069704,100407.044782
1196,101319.547304,100488.495715,100095.377369,98846.532087,99929.697922,99804.151136,101197.438340,98978.499713,100768.580878,100674.503019,99887.921065,100308.605924,99541.311873,99895.092679,99260.693678,101057.586537,99850.288865,99702.579939,99576.344058,100217.250313,99499.221145,100644.604475,99093.739347,100554.727919,100748.445290,99976.564929,101460.111439,99717.615667,100599.259168,100874.437880,100228.863671,100191.132047,100132.237305,99856.324550,100319.765239,99116.303072,99365.508990,99725.193051,99876.756547,101470.386367,...,100901.828036,101468.565110,99769.019922,100768.300781,99913.391598,100783.366089,100108.543034,100624.541931,100228.223969,100146.971615,100312.148281,100000.840384,100939.757794,100853.006153,99680.699974,100333.843204,99271.905760,99268.668278,99574.802897,100565.697306,100308.888747,99477.293536,100067.692999,100220.416430,100188.839891,100112.967258,99985.782459,100710.970968,99229.223602,100659.181004,101288.052212,99520.068467,100737.559367,99280.051412,100353.499413,100250.913714,99167.552034,100769.839760,100105.757395,98727.890915
1197,99953.187978,101242.634686,100587.128968,100491.564198,99522.330773,100902.113929,100764.063039,99674.705503,99692.495012,100692.753552,100700.464336,100520.913778,100369.436702,100493.763348,99954.977808,100285.897563,99646.336018,99927.632207,100055.688908,100566.816997,99614.505369,100666.921674,101040.287511,99960.308699,100692.320119,100103.651065,100443.632412,99746.241013,100863.134973,99693.524401,101084.536572,99830.055348,100465.451854,101003.388846,100134.876237,99564.825943,100933.170891,99388.712707,100551.616178,100648.156017,...,100831.811151,100081.366246,100025.888454,99958.242860,100203.934431,101162.503642,99014.246873,100092.993586,100269.852812,100102.225877,99974.064453,100807.480145,99755.257238,100150.631158,99744.983807,100876.133536,99782.304813,100815.382829,100090.272535,99807.853898,100232.882064,100035.913810,101328.615219,100585.429889,99623.934797,100470.165533,100718.892517,99684.622726,100027.510978,101160.782733,101044.754220,99206.459921,100663.639340,99765.307361,99477.311880,99439.680163,99389.923647,101229.103360,101825.339897,99620.148008
1198,100513.202168,100429.067452,100266.193291,99541.824001,99972.519193,99564.903384,100585.146877,100079.901008,99974.301471,99025.124348,100524.639283,100305.032276,100493.398938,100095.128888,100920.379608,101320.730808,99870.421707,100927.527914,100642.564472,98673.690440,100630.154351,99534.388027,99677.889449,100595.862768,100032.553862,100588.340166,100361.407369,100286.973118,101848.047186,100902.528165,10048